# 0/1背包问题的分支限界算法设计与实现
**姓名：邓康**
**学号：2025439148**
**学院：天津大学福州国际联合学院**
**专业：电子信息**

**目标**：在 0/1 背包（每个物品要么选要么不选）中，设计基于分支限界的经典方法。

---

## 算法 A：Branch-and-Bound Basic（基础分支限界）

- **思路**：采用深度优先搜索（DFS）策略，在递归树中枚举每个物品的选（1）与不选（0）分支。在每一个树节点，算法计算一个分数背包上界：假设后续物品可以按价值密度$（\rho_i = \frac{v_i}{w_i}）$
降序部分装入，由此得到的最大可能价值

- **优点**：能保证找到全局最优解。分数背包上界计算高效，提供了有效的剪枝依据

- **缺点**：搜索策略（DFS）相对盲目，可能长时间探索潜力较低的子树。剪枝仅依赖一个较宽松的上界，在复杂实例中搜索树规模仍可能爆炸性增长，求解效率不稳定。

---

## 算法 B：Enhanced Branch-and-Bound via Probing & Node Selection（探测和最佳界限节点选择优化）

- **思路**：在正式对一个变量（物品）进行分支（设为0或1）之前，进行预探索,临时固定该变量，快速评估其两个子问题的线性松弛上界。在节点选择上选择优先级最高（即上界最紧、理论上潜力最大）的节点。
  $$
  P(node)=UB_{frac}(node)
  $$

- **作用**：改变了基础版本中深度优先的“盲目”搜索顺序。该策略能引导算法最快地找到高质量可行解，从而迅速提升全局下界，配合探测技术实现更早、更有效的剪

- **理论性质（这里只做提示，证明放在'理论分析'章节）**：“探测”与“基于上界的节点选择”相结合，能系统性减少为证明最优性所需探索的搜索树规模（节点数）。这直接转化为更短的求解时间，尤其是在那些传统方法需要大量回溯的“难”实例上。

In [1]:
# -*- coding: utf-8 -*-
from dataclasses import dataclass, field
from typing import List, Tuple, Optional
import heapq
from copy import deepcopy

@dataclass
class Item:
    """物品结构"""
    w: int  # 重量
    v: int  # 价值
    idx: int  # 原始索引
    
    @property
    def density(self) -> float:
        """价值密度"""
        return self.v / self.w if self.w > 0 else 0

@dataclass(order=True)
class BBNode:
    """
    分支限界树节点结构（用于增强版优先队列）
    按上界值降序排列，Python的heapq是最小堆，因此用负值实现最大堆效果
    """
    neg_bound: float = field(compare=True)  # 负的上界值（用于优先队列排序）
    level: int = field(compare=False)       # 决策层级（已考虑的物品数）
    value: int = field(compare=False)       # 当前总价值
    weight: int = field(compare=False)      # 当前总重量
    taken: List[int] = field(compare=False) # 已选物品索引
    bound: float = field(compare=False)     # 上界值
    
    def __post_init__(self):
        """确保taken列表是副本，避免引用问题"""
        self.taken = self.taken.copy()

# ---------- 辅助函数 ----------
def fractional_knapsack_bound(
    items: List[Item],
    capacity: int,
    level: int,
    current_value: int,
    current_weight: int
) -> Tuple[float, float]:
    """
    计算分数背包上界（贪心松弛上界）
    
    返回: (上界值, 分数部分的价值)
    原理:
    1. 从第level个物品开始（0-indexed，假设items已按密度降序排序）
    2. 尽可能装入完整物品
    3. 剩余容量装入第一个无法完整装入物品的一部分（按密度比例）
    """
    if current_weight > capacity:
        return -float('inf'), 0.0  # 不可行节点
    
    bound_value = float(current_value)
    remaining_capacity = capacity - current_weight
    
    # 遍历剩余物品
    for i in range(level, len(items)):
        if items[i].w <= remaining_capacity:
            # 可以完整装入
            bound_value += items[i].v
            remaining_capacity -= items[i].w
        else:
            # 只能装入一部分
            fraction = remaining_capacity / items[i].w
            fractional_value = items[i].v * fraction
            bound_value += fractional_value
            return bound_value, fractional_value
    
    return bound_value, 0.0  # 所有物品都能完整装入

def sort_items_by_density(items: List[Item]) -> List[Item]:
    """按价值密度降序排序物品（用于上界计算）"""
    return sorted(items, key=lambda x: x.v / x.w, reverse=True)

# ---------- 算法一：基础分支限界法 (bb_basic) ----------
def branch_and_bound_basic(
    items: List[Item],
    capacity: int
) -> Tuple[int, List[int], Dict[str, int]]:
    """
    基础分支限界法（深度优先搜索 + 分数背包上界剪枝）
    返回: (最优价值, 最优解物品索引列表, 性能指标字典)
    
    算法特点:
    1. 深度优先搜索（栈实现）
    2. 物品按价值密度降序预处理，以获得更紧的上界
    3. 使用分数背包上界进行剪枝
    
    性能指标:
    - nodes_generated: 生成的节点总数
    - nodes_explored: 实际探索的节点数
    - first_opt_node: 首次找到最优解的节点编号
    """
    # 按价值密度降序排序（提高上界质量）
    sorted_items = sort_items_by_density(items)
    n = len(sorted_items)
    
    # 性能指标记录
    metrics = {
        'nodes_generated': 1,  # 初始节点
        'nodes_explored': 0,
        'first_opt_node': 0
    }
    
    # 全局最优解记录
    best_value = 0
    best_taken = []
    
    # 使用栈进行DFS（每个元素: (level, value, weight, taken)）
    stack = [(0, 0, 0, [])]  # 从第0层开始
    
    while stack:
        level, value, weight, taken = stack.pop()
        metrics['nodes_explored'] += 1
        
        # 检查是否到达叶子节点
        if level == n:
            if value > best_value and weight <= capacity:
                best_value = value
                best_taken = taken
                if metrics['first_opt_node'] == 0:
                    metrics['first_opt_node'] = metrics['nodes_explored']
            continue
        
        # 计算上界（包括当前节点的价值）
        bound, _ = fractional_knapsack_bound(
            sorted_items, capacity, level, value, weight
        )
        
        # 剪枝：如果上界不大于当前最优解，放弃该分支
        if bound <= best_value:
            continue
        
        # 处理当前物品
        current_item = sorted_items[level]
        
        # 分支1：不选当前物品（左子树）
        stack.append((level + 1, value, weight, taken.copy()))
        metrics['nodes_generated'] += 1
        
        # 分支2：选当前物品（右子树）- 需检查可行性
        new_weight = weight + current_item.w
        if new_weight <= capacity:
            new_taken = taken + [current_item.idx]  # 记录原始索引
            new_value = value + current_item.v
            stack.append((level + 1, new_value, new_weight, new_taken))
            metrics['nodes_generated'] += 1
    
    return best_value, best_taken, metrics

# ---------- 算法二：增强分支限界法 (bb_enhanced) ----------
def branch_and_bound_enhanced(
    items: List[Item],
    capacity: int
) -> Tuple[int, List[int], Dict[str, int]]:
    """
    增强分支限界法（探测技术 + 最佳上界优先）
    
    返回: (最优价值, 最优解物品索引列表, 性能指标字典)
    
    核心优化：
    1. 探测技术（Probing）：分支前评估子问题潜力，提前剪枝无望分支
    2. 最佳界限节点选择（Best-Bound Node Selection）：总是扩展上界最高的节点
    
    性能指标:
    - nodes_generated: 生成的节点总数
    - nodes_explored: 实际探索的节点数
    - first_opt_node: 首次找到最优解的节点编号
    - nodes_pruned_by_probing: 被探测技术剪枝的节点数
    """
    # 按价值密度降序排序
    sorted_items = sort_items_by_density(items)
    n = len(sorted_items)
    
    # 性能指标记录
    metrics = {
        'nodes_generated': 1,  # 初始节点
        'nodes_explored': 0,
        'first_opt_node': 0,
        'nodes_pruned_by_probing': 0
    }
    
    # 全局最优解记录
    best_value = 0
    best_taken = []
    
    # 活节点表（优先队列）- 实现最佳界限优先
    priority_queue = []
    
    # 创建初始节点（包含上界信息）
    initial_bound, _ = fractional_knapsack_bound(
        sorted_items, capacity, 0, 0, 0
    )
    heapq.heappush(
        priority_queue,
        BBNode(
            neg_bound=-initial_bound,  # 负值实现最大堆（优先取上界高的）
            level=0,
            value=0,
            weight=0,
            taken=[],
            bound=initial_bound
        )
    )
    
    while priority_queue:
        # 1：最佳界限节点选择
        current_node = heapq.heappop(priority_queue)
        metrics['nodes_explored'] += 1
        
        level = current_node.level
        value = current_node.value
        weight = current_node.weight
        taken = current_node.taken
        current_bound = current_node.bound
        
        # 检查是否到达叶子节点
        if level == n:
            if value > best_value and weight <= capacity:
                best_value = value
                best_taken = taken
                if metrics['first_opt_node'] == 0:
                    metrics['first_opt_node'] = metrics['nodes_explored']
            continue
        
        current_item = sorted_items[level]
        
        # 2：探测技术（Probing）
        # 评估左分支（不选当前物品）
        left_bound, _ = fractional_knapsack_bound(
            sorted_items, capacity, level + 1, value, weight
        )
        
        # 评估右分支（选当前物品）- 仅在可行时
        new_weight = weight + current_item.w
        right_bound = -float('inf')
        right_feasible = new_weight <= capacity
        
        if right_feasible:
            new_value = value + current_item.v
            right_bound, _ = fractional_knapsack_bound(
                sorted_items, capacity, level + 1, new_value, new_weight
            )
        
        # 基于探测结果的剪枝决策
        max_child_bound = max(left_bound, right_bound) if right_feasible else left_bound
        
        if max_child_bound <= best_value:
            # 所有子分支都不可能产生更优解，完全剪枝该节点
            metrics['nodes_pruned_by_probing'] += 1
            continue
        
        # 创建子节点，按上界值加入优先队列
        # 先创建右分支（选当前物品）
        if right_feasible and right_bound > best_value:
            new_taken = taken + [current_item.idx]
            new_value = value + current_item.v
            
            right_node = BBNode(
                neg_bound=-right_bound,
                level=level + 1,
                value=new_value,
                weight=new_weight,
                taken=new_taken,
                bound=right_bound
            )
            heapq.heappush(priority_queue, right_node)
            metrics['nodes_generated'] += 1
        
        # 创建左分支（不选当前物品）
        if left_bound > best_value:
            left_node = BBNode(
                neg_bound=-left_bound,
                level=level + 1,
                value=value,
                weight=weight,
                taken=taken.copy(),
                bound=left_bound
            )
            heapq.heappush(priority_queue, left_node)
            metrics['nodes_generated'] += 1
    
    return best_value, best_taken, metrics

## 2. 理论分析（两种分支限界方法）

**问题**
给定物品集合 $\{(w_i,v_i)\}_{i=1}^n$ 与容量 $C$，选择子集使得总重量 $\le C$ 且总价值最大（0/1 背包）。

---

### 方法 A：Branch-and-Bound Basic（基础分支限界法）
- **设计思想**：采用深度优先搜索策略，在递归树中枚举每个物品的选（1）与不选（0）分支。在每一个树节点，算法计算一个分数背包上界：假设后续物品可以按价值密度$\rho_i = v_i/w_i$降序部分装入，由此得到的最大可能价值。若某节点的上界不大于当前全局最优解，则剪除该分支。
- **时间复杂度**：最坏情况下需探索指数级数量的节点$O(2^n)$。通过上界剪枝，实际探索的节点数通常远小于理论最坏情况。每次节点处理需计算分数背包上界，其复杂度为$O(n)$,总时间复杂度上界为$O(2^n⋅n)$。
- **空间复杂度**：深度优先搜索使用栈最多同时存储$O(n)$个节点。
- **性能保证**：分支限界法是一种精确算法，能够保证找到全局最优解。
- **关键瓶颈**：
  1) **搜索策略的盲目性**：深度优先搜索可能长时间陷入非最优的分支，导致延迟找到高质量可行解，从而影响全局剪枝效率；
  2) **上界紧度有限**：分数背包上界虽比线性松弛更紧，但对某些结构化实例（如物品价值与重量高度相关）仍然不够紧，导致剪枝效果不足；
  3) **节点扩展顺序固定**：按物品顺序分支（先考虑不选，再考虑选），缺乏对问题特征的适应性调整。

---

### 方法 B：Enhanced Branch-and-Bound via Probing & Node Selection（探测和最佳界限节点选择优化）
- **设计思想**：在正式对一个变量（物品）进行分支之前，进行预探索。使用优先队列管理活节点，总是选择上界值最高的节点进行扩展。
- **时间复杂度**：最坏情况下仍为$O(2^n)$，但期望通过更强的剪枝（探测）和更优的节点选择，大幅减少实际探索的节点数。
- **空间复杂度**：优先队列可能存储大量节点，最坏情况下空间复杂度$O(2^n)$。
- **性能保证**：增强分支限界法同样是一种精确算法，保证找到全局最优解。优化组件仅改变搜索顺序和剪枝时机，不改变算法完备性。
(i) 搜索效率提升：论文通过大量实验证明，"探测"与"基于上界的节点选择"相结合，能系统性减少为证明最优性所需探索的搜索树规模（节点数）；
(ii) 更快找到高质量解：最佳界限优先策略能引导算法迅速找到优质可行解，从而快速提升全局下界，加速后续剪枝；
(iii) 适应性：探测技术能根据实例特征动态调整搜索方向，对传统方法难以处理的"贪心陷阱"类实例效果显著。
- **关键瓶颈**：
  1) **空间复杂度**：最佳界限优先策略需使用优先队列维护所有活节点，在最坏情况下队列规模可达 $O(2^n)$；
  2) **探测计算开销**：每个节点的分支前需额外计算两次完整的上界评估（左、右分支），若探测剪枝效果不明显，额外计算可能成为主要开销；
  3) **浮点数精度问题**：分数背包上界涉及浮点运算，可能因数值精度误差导致错误的剪枝决策。

---

### 比较小结
- **复杂度**：两者最坏时间均为$O(2^n⋅n)$，但增强版通过前沿优化期望大幅减少实际探索节点数；
- **保证**：两者均为精确算法，保证找到最优解;
- **适用性**：增强版在大多数实例上，尤其是传统方法需大量回溯的"难"实例，应表现出更快的求解速度与更少的节点探索。


### 参考文献
- 1.Land, A. H. & Doig, A. G. (1960). An Automatic Method of Solving Discrete Programming Problems. Econometrica, 28, pp. 497-520.
- 2.Mans, Bernard & Roucairol, Catherine. (1996). Performance of parallel branch-and-bound algorithms with best-first search. Discrete Applied Mathematics. 66. 57-74. 10.     1016/0166-218X(94)00137-3. 
- 3.David R. Morrison, Sheldon H. Jacobson, Jason J. Sauppe, and Edward C. Sewell. 2016. Branch-and-bound algorithms. Discret. Optim. 19, C (February 2016), 79–102.doi.org/10.1016/j.disopt.2016.01.005
- 4.Forget, N. and Parragh, S. N., “Enhancing Branch-and-Bound for Multi-Objective 0-1 Programming”, <i>arXiv e-prints</i>, Art. no. arXiv:2210.05385, 2022. doi:10.48550/arXiv.2210.05385.


## 3. 实验设计（只围绕两种算法：Branch-and-Bound Basic 与 Enhanced Branch-and-Bound via Probing & Node Selection）

**目标**
系统比较两种贪心方法在不同数据特性与规模下的运行表现，确保结果具备统计意义与可复现性。

**实例族（≥5 类，覆盖结构化/随机与不同分布）**
1) **Uncorrelated**：重量与价值独立均匀；
2) **WeaklyCorr**：弱相关（$v \approx w + \text{noise}$）；
3) **StronglyCorr**：强相关（$v = w + \Delta$）；
4) **GreedyTrap**：大量小而高密度物品 + 单个价值极高但密度略差的重物品；
5) **SubsetSum**：子集和问题（价值=重量，搜索树对称，最优解通常位于边界）
6) **ProbingBenefit**：探测优势类（一个“决定性的”高价值物品，一组“诱饵”物品：密度略高但总价值有限，一组“填充”物品：密度很低，价值有限）。

**规模与重复**
- 每类 **20 个样本 × ≥5 次随机重复**（本文设为 5），总计 ≥100 个样本/类；
- 启用容量维度 DP（$O(nC)$）获得 OPT；
- 固定随机种子方案（按类名、repeat、sample 组合）以保证可复现。

**记录指标（本节只"生成与运行"，汇总统计留到下一节）**
- **运行时间**（每次调用的 wall-clock）；
- **生成节点数**（算法创建的所有节点，包括被剪枝的）；
- **探索节点数**（算法实际从栈/队列中弹出并处理的节点总数）；
- **剪枝比例**：(1 - 探索节点数/生成节点数)×100%其中"生成节点数"=所有创建的节点（包括被剪枝的）；
- **首次最优节点**：算法首次达到全局最优解时已探索的节点数
- **解值**（两算法的总价值与DP）
- 将原始结果表保存为 CSV，便于后续计算均值/方差与显著性检验。

**复现实验的关键点**
- 统一容量比例（例如 $C = 0.5 \sum w_i$），避免无意义的过松或过紧；
- 采用固定的随机种子拼接策略，保证"同一 (类, repeat, sample)"跨算法完全一致；

In [21]:
# -*- coding: utf-8 -*-
# 第三节：实验设计 —— 可直接运行的实现代码
"""
分支限界法对比实验：基础版本 vs 优化版本
实验框架参考：稳定哈希、实例生成器、可复现性
"""

import math
import random
import time
import os
import heapq
import tracemalloc  # 添加内存测量
from dataclasses import dataclass, field
from typing import List, Tuple, Dict, Callable
import numpy as np
import pandas as pd
import zlib

# ---------- 稳定哈希（用于可复现的随机种子） ----------
def stable_hash_str(*parts) -> int:
    """跨平台/跨进程稳定的32位哈希，用于生成可复现的随机种子。"""
    s = "|".join(str(p) for p in parts)
    return zlib.adler32(s.encode("utf-8")) & 0xffffffff

# ---------- 随机与输出设置 ----------
random.seed(42)
np.random.seed(42)
OUTDIR = "outputs_branch_bound"
os.makedirs(OUTDIR, exist_ok=True)

# ---------- 数据模型定义 ----------
@dataclass
class Item:
    """物品结构"""
    w: int  # 重量
    v: int  # 价值
    idx: int  # 原始索引
    
    @property
    def density(self) -> float:
        """价值密度"""
        return self.v / self.w if self.w > 0 else 0

@dataclass(order=True)
class BBNode:
    """
    分支限界树节点结构（用于增强版优先队列）
    按上界值降序排列，Python的heapq是最小堆，因此用负值实现最大堆效果
    """
    neg_bound: float = field(compare=True)  # 负的上界值（用于优先队列排序）
    level: int = field(compare=False)       # 决策层级（已考虑的物品数）
    value: int = field(compare=False)       # 当前总价值
    weight: int = field(compare=False)      # 当前总重量
    taken: List[int] = field(compare=False) # 已选物品索引
    bound: float = field(compare=False)     # 上界值
    
    def __post_init__(self):
        """确保taken列表是副本，避免引用问题"""
        self.taken = self.taken.copy()

# ---------- 辅助函数 ----------
def fractional_knapsack_bound(
    items: List[Item],
    capacity: int,
    level: int,
    current_value: int,
    current_weight: int
) -> Tuple[float, float]:
    """
    计算分数背包上界（贪心松弛上界）
    
    返回: (上界值, 分数部分的价值)
    原理:
    1. 从第level个物品开始（0-indexed，假设items已按密度降序排序）
    2. 尽可能装入完整物品
    3. 剩余容量装入第一个无法完整装入物品的一部分（按密度比例）
    """
    if current_weight > capacity:
        return -float('inf'), 0.0  # 不可行节点
    
    bound_value = float(current_value)
    remaining_capacity = capacity - current_weight
    
    # 遍历剩余物品
    for i in range(level, len(items)):
        if items[i].w <= remaining_capacity:
            # 可以完整装入
            bound_value += items[i].v
            remaining_capacity -= items[i].w
        else:
            # 只能装入一部分
            fraction = remaining_capacity / items[i].w
            fractional_value = items[i].v * fraction
            bound_value += fractional_value
            return bound_value, fractional_value
    
    return bound_value, 0.0  # 所有物品都能完整装入

def sort_items_by_density(items: List[Item]) -> List[Item]:
    """按价值密度降序排序物品（用于上界计算）"""
    return sorted(items, key=lambda x: x.v / x.w, reverse=True)

# ---------- 动态规划验证 ----------
def dp_optimal(items: List[Item], C: int) -> Tuple[int, List[int]]:
    """容量维度 DP，O(nC)。规模过大时仅返回值，不回溯。"""
    n = len(items)
    dp = [0]*(C+1)
    if (n+1)*(C+1) <= 2_000_000:
        take = [[False]*(C+1) for _ in range(n)]
        for i, it in enumerate(items):
            w, v = it.w, it.v
            for c in range(C, w-1, -1):
                if dp[c-w] + v > dp[c]:
                    dp[c] = dp[c-w] + v
                    take[i][c] = True
        c = max(range(C+1), key=lambda x: dp[x])
        chosen = []
        for i in range(n-1, -1, -1):
            if take[i][c]:
                chosen.append(items[i].idx)
                c -= items[i].w
        return max(dp), chosen[::-1]
    else:
        for it in items:
            w, v = it.w, it.v
            for c in range(C, w-1, -1):
                if dp[c-w] + v > dp[c]:
                    dp[c] = dp[c-w] + v
        return max(dp), []

# ---------- 算法一：基础分支限界法 (bb_basic) ----------
def branch_and_bound_basic(
    items: List[Item],
    capacity: int
) -> Tuple[int, List[int], Dict[str, int]]:
    """
    基础分支限界法（深度优先搜索 + 分数背包上界剪枝）
    返回: (最优价值, 最优解物品索引列表, 性能指标字典)
    
    算法特点:
    1. 深度优先搜索（栈实现）
    2. 物品按价值密度降序预处理，以获得更紧的上界
    3. 使用分数背包上界进行剪枝
    
    性能指标:
    - nodes_generated: 生成的节点总数
    - nodes_explored: 实际探索的节点数
    - first_opt_node: 首次找到最优解的节点编号
    """
    # 按价值密度降序排序（提高上界质量）
    sorted_items = sort_items_by_density(items)
    n = len(sorted_items)
    
    # 性能指标记录
    metrics = {
        'nodes_generated': 1,  # 初始节点
        'nodes_explored': 0,
        'first_opt_node': 0
    }
    
    # 全局最优解记录
    best_value = 0
    best_taken = []
    
    # 使用栈进行DFS（每个元素: (level, value, weight, taken)）
    stack = [(0, 0, 0, [])]  # 从第0层开始
    
    while stack:
        level, value, weight, taken = stack.pop()
        metrics['nodes_explored'] += 1
        
        # 检查是否到达叶子节点
        if level == n:
            if value > best_value and weight <= capacity:
                best_value = value
                best_taken = taken
                if metrics['first_opt_node'] == 0:
                    metrics['first_opt_node'] = metrics['nodes_explored']
            continue
        
        # 计算上界（包括当前节点的价值）
        bound, _ = fractional_knapsack_bound(
            sorted_items, capacity, level, value, weight
        )
        
        # 剪枝：如果上界不大于当前最优解，放弃该分支
        if bound <= best_value:
            continue
        
        # 处理当前物品
        current_item = sorted_items[level]
        
        # 分支1：不选当前物品（左子树）
        stack.append((level + 1, value, weight, taken.copy()))
        metrics['nodes_generated'] += 1
        
        # 分支2：选当前物品（右子树）- 需检查可行性
        new_weight = weight + current_item.w
        if new_weight <= capacity:
            new_taken = taken + [current_item.idx]  # 记录原始索引
            new_value = value + current_item.v
            stack.append((level + 1, new_value, new_weight, new_taken))
            metrics['nodes_generated'] += 1
    
    return best_value, best_taken, metrics

# ---------- 算法二：增强分支限界法 (bb_enhanced) ----------
def branch_and_bound_enhanced(
    items: List[Item],
    capacity: int
) -> Tuple[int, List[int], Dict[str, int]]:
    """
    增强分支限界法（探测技术 + 最佳上界优先）
    
    返回: (最优价值, 最优解物品索引列表, 性能指标字典)
    
    核心优化：
    1. 探测技术（Probing）：分支前评估子问题潜力，提前剪枝无望分支
    2. 最佳界限节点选择（Best-Bound Node Selection）：总是扩展上界最高的节点
    
    性能指标:
    - nodes_generated: 生成的节点总数
    - nodes_explored: 实际探索的节点数
    - first_opt_node: 首次找到最优解的节点编号
    - nodes_pruned_by_probing: 被探测技术剪枝的节点数
    """
    # 按价值密度降序排序
    sorted_items = sort_items_by_density(items)
    n = len(sorted_items)
    
    # 性能指标记录
    metrics = {
        'nodes_generated': 1,  # 初始节点
        'nodes_explored': 0,
        'first_opt_node': 0,
        'nodes_pruned_by_probing': 0
    }
    
    # 全局最优解记录
    best_value = 0
    best_taken = []
    
    # 活节点表（优先队列）- 实现最佳界限优先
    priority_queue = []
    
    # 创建初始节点（包含上界信息）
    initial_bound, _ = fractional_knapsack_bound(
        sorted_items, capacity, 0, 0, 0
    )
    heapq.heappush(
        priority_queue,
        BBNode(
            neg_bound=-initial_bound,  # 负值实现最大堆（优先取上界高的）
            level=0,
            value=0,
            weight=0,
            taken=[],
            bound=initial_bound
        )
    )
    
    while priority_queue:
        # 1：最佳界限节点选择
        current_node = heapq.heappop(priority_queue)
        metrics['nodes_explored'] += 1
        
        level = current_node.level
        value = current_node.value
        weight = current_node.weight
        taken = current_node.taken
        current_bound = current_node.bound
        
        # 检查是否到达叶子节点
        if level == n:
            if value > best_value and weight <= capacity:
                best_value = value
                best_taken = taken
                if metrics['first_opt_node'] == 0:
                    metrics['first_opt_node'] = metrics['nodes_explored']
            continue
        
        current_item = sorted_items[level]
        
        # 2：探测技术（Probing）
        # 评估左分支（不选当前物品）
        left_bound, _ = fractional_knapsack_bound(
            sorted_items, capacity, level + 1, value, weight
        )
        
        # 评估右分支（选当前物品）- 仅在可行时
        new_weight = weight + current_item.w
        right_bound = -float('inf')
        right_feasible = new_weight <= capacity
        
        if right_feasible:
            new_value = value + current_item.v
            right_bound, _ = fractional_knapsack_bound(
                sorted_items, capacity, level + 1, new_value, new_weight
            )
        
        # 基于探测结果的剪枝决策
        max_child_bound = max(left_bound, right_bound) if right_feasible else left_bound
        
        if max_child_bound <= best_value:
            # 所有子分支都不可能产生更优解，完全剪枝该节点
            metrics['nodes_pruned_by_probing'] += 1
            continue
        
        # 创建子节点，按上界值加入优先队列
        # 先创建右分支（选当前物品）
        if right_feasible and right_bound > best_value:
            new_taken = taken + [current_item.idx]
            new_value = value + current_item.v
            
            right_node = BBNode(
                neg_bound=-right_bound,
                level=level + 1,
                value=new_value,
                weight=new_weight,
                taken=new_taken,
                bound=right_bound
            )
            heapq.heappush(priority_queue, right_node)
            metrics['nodes_generated'] += 1
        
        # 创建左分支（不选当前物品）
        if left_bound > best_value:
            left_node = BBNode(
                neg_bound=-left_bound,
                level=level + 1,
                value=value,
                weight=weight,
                taken=taken.copy(),
                bound=left_bound
            )
            heapq.heappush(priority_queue, left_node)
            metrics['nodes_generated'] += 1
    
    return best_value, best_taken, metrics

# ---------- 算法字典 ----------
ALGOS: Dict[str, Callable[[List[Item], int], Tuple[int, List[int], Dict]]] = {
    "bb_basic": branch_and_bound_basic,
    "bb_enhanced": branch_and_bound_enhanced,
}

# ---------- 实例生成器（6种类型） ----------
C_BASE_DEFAULT = 1000

def gen_uncorrelated(n:int, seed:int, cap_ratio=0.5, C_base:int=C_BASE_DEFAULT):
    """Uncorrelated: 重量价值独立均匀分布"""
    rng = random.Random(seed)
    C = int(C_base * cap_ratio)
    items = []
    for i in range(n):
        w = rng.randint(1, 100)
        v = rng.randint(1, 100)
        items.append(Item(w=w, v=v, idx=i))
    return items, C

def gen_weakly_correlated(n:int, seed:int, cap_ratio=0.5, C_base:int=C_BASE_DEFAULT):
    """WeaklyCorr: 弱相关 (v ≈ w + noise)"""
    rng = random.Random(seed)
    C = int(C_base * cap_ratio)
    items = []
    for i in range(n):
        w = rng.randint(1, 100)
        v = max(1, w + rng.randint(-10, 10))
        items.append(Item(w=w, v=v, idx=i))
    return items, C

def gen_strongly_correlated(n:int, seed:int, cap_ratio=0.5, C_base:int=C_BASE_DEFAULT):
    """StronglyCorr: 强相关 (v = w + Δ)"""
    rng = random.Random(seed)
    C = int(C_base * cap_ratio)
    items = []
    for i in range(n):
        w = rng.randint(1, 100)
        v = w + 10  # Δ = 10
        items.append(Item(w=w, v=v, idx=i))
    return items, C

def gen_greedy_trap(n:int, seed:int, cap_ratio=0.5, C_base:int=C_BASE_DEFAULT):
    """GreedyTrap: 贪心陷阱实例"""
    rng = random.Random(seed)
    C = int(C_base * cap_ratio)
    items = []
    # 第一个物品：重量大但密度极高
    items.append(Item(w=100, v=1000, idx=0))
    # 其他物品：重量小但密度略低
    for i in range(1, n):
        w = rng.randint(1, 10)
        v = w * 8  # 密度8
        items.append(Item(w=w, v=v, idx=i))
    return items, C

def gen_subset_sum(n:int, seed:int, cap_ratio=0.5, C_base:int=C_BASE_DEFAULT):
    """SubsetSum: 子集和问题 (v = w)"""
    rng = random.Random(seed)
    C = int(C_base * cap_ratio)
    items = []
    for i in range(n):
        w = rng.randint(1, 100)
        items.append(Item(w=w, v=w, idx=i))
    return items, C

def gen_probing_benefit(n:int, seed:int, cap_ratio=0.5, C_base:int=C_BASE_DEFAULT):
    """ProbingBenefit: 专门突出探测优势的实例"""
    rng = random.Random(seed)
    C = int(C_base * cap_ratio)
    items = []
    # 前半部分：高密度小物品
    for i in range(n // 2):
        w = rng.randint(1, 5)
        v = w * 20  # 高密度
        items.append(Item(w=w, v=v, idx=i))
    # 后半部分：低密度大物品
    for i in range(n // 2, n):
        w = rng.randint(50, 100)
        v = w // 2  # 低密度
        items.append(Item(w=w, v=v, idx=i))
    return items, C

GENS = {
    "Uncorrelated": gen_uncorrelated,
    "WeaklyCorr": gen_weakly_correlated,
    "StronglyCorr": gen_strongly_correlated,
    "GreedyTrap": gen_greedy_trap,
    "SubsetSum": gen_subset_sum,
    "ProbingBenefit": gen_probing_benefit,
}

# ---------- 单实例评测 ----------
def run_instance(items: List[Item], C: int, timeout: int = 300):
    """在单个实例上评测两种分支限界算法，返回记录"""
    n = len(items)
    total_w = sum(it.w for it in items)
    
    # 获取最优解（用于验证）
    t0 = time.perf_counter()
    if n <= 30 and C <= 10000:  # 小规模使用DP验证
        opt_val, opt_taken = dp_optimal(items, C)
        opt_known = True
    else:
        opt_val = 0
        opt_taken = []
        opt_known = False
    opt_time = time.perf_counter() - t0
    
    rows = []
    for algo_name, algo_func in ALGOS.items():
        try:
            # 设置超时机制
            start_time = time.perf_counter()
            
            # 开始内存追踪
            tracemalloc.start()
            
            # 运行算法
            val, taken, metrics = algo_func(items, C)
            exec_time = time.perf_counter() - start_time
            
            # 获取内存使用情况
            current, peak = tracemalloc.get_traced_memory()
            tracemalloc.stop()
            
            # 检查超时
            if timeout > 0 and exec_time > timeout:
                raise TimeoutError(f"Algorithm {algo_name} exceeded timeout")
            
            # 计算剪枝比例
            pruning_ratio = (1 - metrics['nodes_explored'] / max(metrics['nodes_generated'], 1)) * 100
            
            # 验证正确性（如果知道最优解）
            correct = False
            if opt_known:
                correct = (val == opt_val)
            
            rows.append({
                "algo": algo_name,
                "value": int(val),
                "runtime_s": float(exec_time),
                "nodes_generated": int(metrics['nodes_generated']),
                "nodes_explored": int(metrics['nodes_explored']),
                "pruning_ratio": float(pruning_ratio),
                "first_opt_node": int(metrics['first_opt_node']),
                "memory_peak_mb": float(peak) / 1024 / 1024,  # 转换为MB
                "opt_known": bool(opt_known),
                "opt_value": int(opt_val) if opt_known else 0,
                "opt_time_s": float(opt_time),
                "correct": bool(correct),
                "capacity": int(C),
                "n": int(n),
                "sum_w": int(total_w),
                "timeout": False,
                "error": None
            })
            
        except TimeoutError as e:
            rows.append({
                "algo": algo_name,
                "value": 0,
                "runtime_s": float(timeout),
                "nodes_generated": 0,
                "nodes_explored": 0,
                "pruning_ratio": 0,
                "first_opt_node": 0,
                "memory_peak_mb": 0,
                "opt_known": bool(opt_known),
                "opt_value": int(opt_val) if opt_known else 0,
                "opt_time_s": float(opt_time),
                "correct": False,
                "capacity": int(C),
                "n": int(n),
                "sum_w": int(total_w),
                "timeout": True,
                "error": str(e)
            })
        except Exception as e:
            rows.append({
                "algo": algo_name,
                "value": 0,
                "runtime_s": 0,
                "nodes_generated": 0,
                "nodes_explored": 0,
                "pruning_ratio": 0,
                "first_opt_node": 0,
                "memory_peak_mb": 0,
                "opt_known": bool(opt_known),
                "opt_value": int(opt_val) if opt_known else 0,
                "opt_time_s": float(opt_time),
                "correct": False,
                "capacity": int(C),
                "n": int(n),
                "sum_w": int(total_w),
                "timeout": False,
                "error": str(e)
            })
    
    return rows

# ---------- 主实验循环 ----------
def run_benchmark(seed_base: int = 2025):
    """运行完整的对比实验"""
    
    # 实验配置
    classes = [
        ("Uncorrelated", dict()),
        ("WeaklyCorr", dict()),
        ("StronglyCorr", dict()),
        ("GreedyTrap", dict()),
        ("SubsetSum", dict()),
        ("ProbingBenefit", dict()),
    ]
    
    n_values = [15, 20, 25, 30]
    SAMPLES_PER_SIZE = 20
    
    rows = []
    
    print(f"配置: {len(classes)}种实例类型 × {len(n_values)}种规模 × {SAMPLES_PER_SIZE}样本")
    print(f"算法: {list(ALGOS.keys())}")
    
    # 运行所有实验
    for cls_name, params in classes:
        gen = GENS[cls_name]
        
        for n in n_values:
            for s in range(SAMPLES_PER_SIZE):
                seed = seed_base + (stable_hash_str(cls_name, n, s) % 10_000_000)
                items, C = gen(n, seed, cap_ratio=0.5)
                res = run_instance(items, C, timeout=300)
                
                for r in res:
                    r.update({"class": cls_name, "sample": s, "n": n, "seed": seed})
                    rows.append(r)
            
    
    # 保存原始结果
    df = pd.DataFrame(rows)
    raw_csv = os.path.join(OUTDIR, "raw_results_branch_bound.csv")
    df.to_csv(raw_csv, index=False)
    
    # 显示原始结果前4行
    print(f"\n[OK] Results saved to: {raw_csv}")
    print("\n原始数据文件前4行:")
    print(df.head(4).to_string())
    
    # 生成简单总结
    summary = (
        df.groupby(["class", "algo"])
        .agg(
            mean_time=("runtime_s", "mean"),
            var_time=("runtime_s", "var"),
            mean_nodes=("nodes_explored", "mean"),
            var_nodes=("nodes_explored", "var"),
            mean_memory=("memory_peak_mb", "mean"),
            cnt=("runtime_s", "size")
        )
        .reset_index()
    )
    
    summary_csv = os.path.join(OUTDIR, "summary_preview_branch_bound.csv")
    summary.to_csv(summary_csv, index=False)
    
    # 显示总结文件前4行
    print(f"\n[OK] Summary saved to: {summary_csv}")
    print("\n总结文件前4行:")
    print(summary.head(4).to_string())
    
    return df

# ---------- 主程序 ----------
if __name__ == "__main__":
    print("分支限界法对比实验：基础版本 vs 优化版本")
    
    try:
        df = run_benchmark()
        
    except Exception as e:
        print(f"实验执行出错: {e}")
        import traceback
        traceback.print_exc()

分支限界法对比实验：基础版本 vs 优化版本（带内存测量）
配置: 6种实例类型 × 4种规模 × 20样本
算法: ['bb_basic', 'bb_enhanced']

[OK] Results saved to: outputs_branch_bound\raw_results_branch_bound.csv

原始数据文件前4行:
          algo  value  runtime_s  nodes_generated  nodes_explored  pruning_ratio  first_opt_node  memory_peak_mb  opt_known  opt_value  opt_time_s  correct  capacity   n  sum_w  timeout error         class  sample     seed
0     bb_basic    647   0.001303               81              81            0.0              16        0.001656       True        647    0.000908     True       500  15    661    False  None  Uncorrelated       0  7673696
1  bb_enhanced    647   0.001750               59              59            0.0              34        0.008522       True        647    0.000908     True       500  15    661    False  None  Uncorrelated       0  7673696
2     bb_basic    713   0.000285               43              43            0.0              16        0.001671       True        713    0.000896     True   

## 4. 性能评估指标（只围绕两种算法：Branch-and-Bound Basic 与 Enhanced Branch-and-Bound via Probing & Node Selection）

**目标** 
从"运行时间，节点探索效率、解质量、收敛速度、内存消耗"五个维度系统评估两种分支限界算法，并给出统计量与置信区间量。

### 评估维度与定义
1) **运行时间（Time）**
- 指标：每次算法调用的 wall-clock（秒），每节点处理时间。
- 统计：均值、方差（或标准差）、95% 置信区间。
- 说明：总运行时间体现算法综合性能，每节点处理时间反映算法实现的优化程度。

2) **节点探索效率**
- 指标： 探索节点数，节点探索比例（Exploration Ratio）：探索节点数/生成节点数 × 100%
- 统计：均值、方差、95% 置信区间。
- 说明：探索节点数衡量算法效率，探索比例越低，说明剪枝越有效

3) **解质量**
- 指标：解质量的精准率；
- 统计：均值、标准差、95% 置信区间
- 说明：精确算法，期望精确率接近100%。

4) **收敛速度**
- 指标：首次找到最优解的节点数，1/首次最优节点数；
- 统计：均值、标准差、95% 置信区间
- 说明：首次找到最优解的节点数越小，收敛速度越快，收敛速度指数（0-1）直接反映收敛快慢。

5) **内存消耗**
- **内存消耗（Memory）**：以 Python `tracemalloc` 记录每次调用的**峰值内存**（字节），统计均值与方差。

> 数据来源：第 3 节已生成 `outputs_branch_bound/raw_results_branch_bound.csv`（每类 ≥20 个样本、≥5 次重复），本节在此基础上计算统计量。

In [28]:
# -*- coding: utf-8 -*-
# 第四节：性能评估指标实现代码
# 分支限界法性能评估与可视化分析
# 基于实验数据 outputs_branch_bound/raw_results_branch_bound.csv
# 五个评估维度：
# 1. 运行时间（Time）
# 2. 节点探索效率
# 3. 解质量
# 4. 收敛速度
# 5. 内存消耗

import os
import math
import numpy as np
import pandas as pd
from scipy import stats

# ---------- 路径与输出 ----------
RAW_PATH = "outputs_branch_bound/raw_results_branch_bound.csv"
OUTDIR = "outputs_branch_bound_metrics"
os.makedirs(OUTDIR, exist_ok=True)

if not os.path.exists(RAW_PATH):
    raise FileNotFoundError(
        f"未发现 {RAW_PATH} 。请先运行实验代码以生成原始结果，再执行本节。"
    )

# 加载数据
df = pd.read_csv(RAW_PATH)

# 数据预处理：过滤有效数据
valid_df = df[(df['error'].isna() | (df['error'] == '')) & (df['timeout'] == False)].copy()

# 计算收敛速度指数和节点处理时间
valid_df['convergence_speed'] = 0.0
mask = valid_df['first_opt_node'] > 0
valid_df.loc[mask, 'convergence_speed'] = 1.0 / valid_df.loc[mask, 'first_opt_node']
valid_df['time_per_node'] = valid_df['runtime_s'] / valid_df['nodes_explored'].clip(lower=1)
valid_df['exploration_ratio'] = valid_df['nodes_explored'] / valid_df['nodes_generated'].clip(lower=1) * 100

# ---------- 统计函数 ----------

def ci95(series: pd.Series) -> float:
    """返回均值的 95% 置信区间半径：1.96*s/sqrt(n)（n>1 时）。"""
    s = float(series.std(ddof=1))
    n = int(series.size)
    return 1.96 * s / math.sqrt(n) if n > 1 else 0.0

def binomial_ci(p, n):
    """计算二项分布的比例置信区间（Wald方法）"""
    if n == 0:
        return 0, 0
    se = math.sqrt(p * (1 - p) / n)
    lower = max(0, p - 1.96 * se)
    upper = min(1, p + 1.96 * se)
    return lower, upper

# ---------- 1. 运行时间分析 ----------

time_stats_rows = []
for (cls, algo, n), sub in valid_df.groupby(["class", "algo", "n"]):
    mean_time = float(sub["runtime_s"].mean())
    std_time = float(sub["runtime_s"].std(ddof=1)) if len(sub) > 1 else 0.0
    ci_time = ci95(sub["runtime_s"])
    
    mean_time_per_node = float(sub["time_per_node"].mean())
    std_time_per_node = float(sub["time_per_node"].std(ddof=1)) if len(sub) > 1 else 0.0
    
    time_stats_rows.append({
        "class": cls, "algo": algo, "n": n,
        "mean_runtime_s": mean_time, "std_runtime": std_time, "ci95_runtime": ci_time,
        "mean_time_per_node": mean_time_per_node, "std_time_per_node": std_time_per_node,
        "count": int(len(sub)),
    })

time_stats_df = pd.DataFrame(time_stats_rows).sort_values(["class", "algo", "n"])
time_stats_path = os.path.join(OUTDIR, "runtime_statistics.csv")
time_stats_df.to_csv(time_stats_path, index=False)

# ---------- 2. 节点探索效率分析 ----------

node_stats_rows = []
for (cls, algo, n), sub in valid_df.groupby(["class", "algo", "n"]):
    mean_explored = float(sub["nodes_explored"].mean())
    std_explored = float(sub["nodes_explored"].std(ddof=1)) if len(sub) > 1 else 0.0
    ci_explored = ci95(sub["nodes_explored"])
    
    mean_generated = float(sub["nodes_generated"].mean())
    std_generated = float(sub["nodes_generated"].std(ddof=1)) if len(sub) > 1 else 0.0
    
    mean_ratio = float(sub["exploration_ratio"].mean())
    std_ratio = float(sub["exploration_ratio"].std(ddof=1)) if len(sub) > 1 else 0.0
    
    node_stats_rows.append({
        "class": cls, "algo": algo, "n": n,
        "mean_nodes_explored": mean_explored, "std_nodes_explored": std_explored, "ci95_nodes_explored": ci_explored,
        "mean_nodes_generated": mean_generated, "std_nodes_generated": std_generated,
        "mean_exploration_ratio": mean_ratio, "std_exploration_ratio": std_ratio,
        "count": int(len(sub)),
    })

node_stats_df = pd.DataFrame(node_stats_rows).sort_values(["class", "algo", "n"])
node_stats_path = os.path.join(OUTDIR, "node_efficiency_statistics.csv")
node_stats_df.to_csv(node_stats_path, index=False)

# ---------- 3. 解质量分析 ----------

accuracy_rows = []
for (cls, algo), sub in valid_df.groupby(["class", "algo"]):
    p = float(sub["correct"].mean())
    n = len(sub)
    lower, upper = binomial_ci(p, n)
    
    accuracy_rows.append({
        "class": cls, "algo": algo,
        "accuracy_rate": p, "ci95_lower": lower, "ci95_upper": upper,
        "count": n,
    })

accuracy_df = pd.DataFrame(accuracy_rows)
accuracy_path = os.path.join(OUTDIR, "solution_quality_statistics.csv")
accuracy_df.to_csv(accuracy_path, index=False)

# ---------- 4. 收敛速度分析 ----------

convergence_rows = []
for (cls, algo, n), sub in valid_df.groupby(["class", "algo", "n"]):
    mean_first_opt = float(sub["first_opt_node"].mean())
    std_first_opt = float(sub["first_opt_node"].std(ddof=1)) if len(sub) > 1 else 0.0
    ci_first_opt = ci95(sub["first_opt_node"])
    
    mean_speed = float(sub["convergence_speed"].mean())
    std_speed = float(sub["convergence_speed"].std(ddof=1)) if len(sub) > 1 else 0.0
    ci_speed = ci95(sub["convergence_speed"])
    
    convergence_rows.append({
        "class": cls, "algo": algo, "n": n,
        "mean_first_opt_node": mean_first_opt, "std_first_opt_node": std_first_opt, "ci95_first_opt_node": ci_first_opt,
        "mean_convergence_speed": mean_speed, "std_convergence_speed": std_speed, "ci95_convergence_speed": ci_speed,
        "count": int(len(sub)),
    })

convergence_df = pd.DataFrame(convergence_rows).sort_values(["class", "algo", "n"])
convergence_path = os.path.join(OUTDIR, "convergence_speed_statistics.csv")
convergence_df.to_csv(convergence_path, index=False)

# ---------- 5. 内存消耗分析 ----------

memory_rows = []
for (cls, algo, n), sub in valid_df.groupby(["class", "algo", "n"]):
    mean_mem = float(sub["memory_peak_mb"].mean())
    std_mem = float(sub["memory_peak_mb"].std(ddof=1)) if len(sub) > 1 else 0.0
    ci_mem = ci95(sub["memory_peak_mb"])
    max_mem = float(sub["memory_peak_mb"].max())
    min_mem = float(sub["memory_peak_mb"].min())
    
    memory_rows.append({
        "class": cls, "algo": algo, "n": n,
        "mean_memory_mb": mean_mem, "std_memory_mb": std_mem, "ci95_memory_mb": ci_mem,
        "max_memory_mb": max_mem, "min_memory_mb": min_mem,
        "count": int(len(sub)),
    })

memory_df = pd.DataFrame(memory_rows).sort_values(["class", "algo", "n"])
memory_path = os.path.join(OUTDIR, "memory_consumption_statistics.csv")
memory_df.to_csv(memory_path, index=False)

# ---------- 统计显著性检验 ----------

basic_df = valid_df[valid_df['algo'] == 'bb_basic']
enhanced_df = valid_df[valid_df['algo'] == 'bb_enhanced']

metrics_to_test = [
    ('runtime_s', '运行时间'),
    ('nodes_explored', '探索节点数'),
    ('memory_peak_mb', '内存消耗'),
    ('convergence_speed', '收敛速度')
]

significance_rows = []
for metric, metric_name in metrics_to_test:
    t_stat, p_value = stats.ttest_ind(
        basic_df[metric], 
        enhanced_df[metric], 
        equal_var=False, 
        nan_policy='omit'
    )
    
    mean_basic = basic_df[metric].mean()
    mean_enhanced = enhanced_df[metric].mean()
    
    if metric in ['runtime_s', 'nodes_explored', 'memory_peak_mb']:
        improvement_pct = (mean_basic - mean_enhanced) / mean_basic * 100
        if metric == 'memory_peak_mb':
            improvement_pct = -improvement_pct
    else:  # convergence_speed
        improvement_pct = (mean_enhanced - mean_basic) / mean_basic * 100
    
    significance_rows.append({
        'metric': metric_name,
        'mean_basic': mean_basic,
        'mean_enhanced': mean_enhanced,
        'improvement_pct': improvement_pct,
        't_statistic': t_stat,
        'p_value': p_value,
        'significant_05': p_value < 0.05,
    })

significance_df = pd.DataFrame(significance_rows)
significance_path = os.path.join(OUTDIR, "statistical_significance_tests.csv")
significance_df.to_csv(significance_path, index=False)

# ---------- 生成综合汇总 ----------

summary_rows = []
for algo in ['bb_basic', 'bb_enhanced']:
    algo_data = valid_df[valid_df['algo'] == algo]
    
    summary_rows.append({
        '算法': algo,
        '平均运行时间(s)': algo_data['runtime_s'].mean(),
        '运行时间标准差': algo_data['runtime_s'].std(),
        '平均节点处理时间(s)': algo_data['time_per_node'].mean(),
        '平均探索节点数': algo_data['nodes_explored'].mean(),
        '探索比例(%)': algo_data['exploration_ratio'].mean(),
        '精确率(%)': algo_data['correct'].mean() * 100,
        '平均首次最优节点数': algo_data['first_opt_node'].mean(),
        '平均收敛速度指数': algo_data['convergence_speed'].mean(),
        '平均内存消耗(MB)': algo_data['memory_peak_mb'].mean(),
        '样本数量': len(algo_data)
    })

summary_df = pd.DataFrame(summary_rows)
summary_path = os.path.join(OUTDIR, "comprehensive_summary.csv")
summary_df.to_csv(summary_path, index=False)

# ---------- 按实例类型汇总 ----------

instance_rows = []
for cls in valid_df['class'].unique():
    for algo in ['bb_basic', 'bb_enhanced']:
        cls_algo_data = valid_df[(valid_df['class'] == cls) & (valid_df['algo'] == algo)]
        
        if len(cls_algo_data) > 0:
            # 计算增强版相对于基础版的改进百分比
            basic_data = valid_df[(valid_df['class'] == cls) & (valid_df['algo'] == 'bb_basic')]
            enhanced_data = valid_df[(valid_df['class'] == cls) & (valid_df['algo'] == 'bb_enhanced')]
            
            if len(basic_data) > 0 and len(enhanced_data) > 0:
                time_improvement = (basic_data['runtime_s'].mean() - enhanced_data['runtime_s'].mean()) / basic_data['runtime_s'].mean() * 100
                nodes_improvement = (basic_data['nodes_explored'].mean() - enhanced_data['nodes_explored'].mean()) / basic_data['nodes_explored'].mean() * 100
            else:
                time_improvement = nodes_improvement = 0
            
            instance_rows.append({
                '实例类型': cls,
                '算法': algo,
                '平均运行时间(s)': cls_algo_data['runtime_s'].mean(),
                '平均探索节点数': cls_algo_data['nodes_explored'].mean(),
                '探索比例(%)': cls_algo_data['exploration_ratio'].mean(),
                '精确率(%)': cls_algo_data['correct'].mean() * 100,
                '时间改进(%)': time_improvement if algo == 'bb_enhanced' else 0,
                '节点改进(%)': nodes_improvement if algo == 'bb_enhanced' else 0,
                '样本数量': len(cls_algo_data)
            })

instance_df = pd.DataFrame(instance_rows)
instance_path = os.path.join(OUTDIR, "instance_type_performance_summary.csv")
instance_df.to_csv(instance_path, index=False)

# ---------- 生成性能提升热力图数据 ----------

heatmap_rows = []
for cls in valid_df['class'].unique():
    basic_data = valid_df[(valid_df['class'] == cls) & (valid_df['algo'] == 'bb_basic')]
    enhanced_data = valid_df[(valid_df['class'] == cls) & (valid_df['algo'] == 'bb_enhanced')]
    
    if len(basic_data) > 0 and len(enhanced_data) > 0:
        # 运行时间提升（正数表示改进）
        time_improvement = (basic_data['runtime_s'].mean() - enhanced_data['runtime_s'].mean()) / basic_data['runtime_s'].mean() * 100
        
        # 节点探索提升
        nodes_improvement = (basic_data['nodes_explored'].mean() - enhanced_data['nodes_explored'].mean()) / basic_data['nodes_explored'].mean() * 100
        
        # 收敛速度提升（正数表示改进）
        convergence_improvement = (enhanced_data['convergence_speed'].mean() - basic_data['convergence_speed'].mean()) / basic_data['convergence_speed'].mean() * 100
        
        heatmap_rows.append({
            '实例类型': cls,
            '时间改进(%)': time_improvement,
            '节点改进(%)': nodes_improvement,
            '收敛改进(%)': convergence_improvement,
            '基础版平均时间(s)': basic_data['runtime_s'].mean(),
            '增强版平均时间(s)': enhanced_data['runtime_s'].mean(),
            '基础版平均节点数': basic_data['nodes_explored'].mean(),
            '增强版平均节点数': enhanced_data['nodes_explored'].mean()
        })

heatmap_df = pd.DataFrame(heatmap_rows)
heatmap_path = os.path.join(OUTDIR, "performance_improvement_heatmap_data.csv")
heatmap_df.to_csv(heatmap_path, index=False)

# ---------- 输出前5个文件的前两行 ----------

files_to_show = [
    ("runtime_statistics.csv", "运行时间统计"),
    ("node_efficiency_statistics.csv", "节点探索效率统计"),
    ("solution_quality_statistics.csv", "解质量统计"),
    ("convergence_speed_statistics.csv", "收敛速度统计"),
    ("memory_consumption_statistics.csv", "内存消耗统计")
]

for filename, description in files_to_show:
    filepath = os.path.join(OUTDIR, filename)
    df = pd.read_csv(filepath)
    print(f"\n{filename} - {description} (前两行):")
    print(df.head(2).to_string(index=False))
    print("-" * 60)



runtime_statistics.csv - 运行时间统计 (前两行):
     class     algo  n  mean_runtime_s  std_runtime  ci95_runtime  mean_time_per_node  std_time_per_node  count
GreedyTrap bb_basic 15        0.000365     0.000039      0.000017            0.000012           0.000001     20
GreedyTrap bb_basic 20        0.000577     0.000078      0.000034            0.000014           0.000002     20
------------------------------------------------------------

node_efficiency_statistics.csv - 节点探索效率统计 (前两行):
     class     algo  n  mean_nodes_explored  std_nodes_explored  ci95_nodes_explored  mean_nodes_generated  std_nodes_generated  mean_exploration_ratio  std_exploration_ratio  count
GreedyTrap bb_basic 15                 31.0                 0.0                  0.0                  31.0                  0.0                   100.0                    0.0     20
GreedyTrap bb_basic 20                 41.0                 0.0                  0.0                  41.0                  0.0                   100

## 5. 数据分析与可视化

**目标**  
对两种算法（Branch-and-Bound Basic 与 Enhanced Branch-and-Bound via Probing & Node Selection）进行系统的数据分析与可视化，满足以下全部要求：

1) **多维性能剖析**  
   - 图表1、2、4、5：展示运行时间、节点效率、收敛速度、内存消耗随问题规模（n）的变化，满足“不同输入规模对性能的影响”。
   - 图表3、8、10、11、12：展示不同实例类型（结构）对算法性能的影响，满足“不同结构对性能的影响”。
   - 图表10：明确展示了增强算法在特定实例中的节点探索优势，可以视为“优势区间图”。
   - 图表15：性能边界总结表。

2) **理论与实验一致性验证**
   - 图表13：专门进行理论与实验一致性验证，包括理论复杂度与实际增长趋势对比，偏差原因分析（每节点处理时间变化反映缓存效应和实现差异）。

3) **显著性统计分析**
   - 图表6：统计显著性检验，包括t检验、p值、改进百分比和显著性标记。
   - 图表9：算法对比汇总，包含置信区间。

4) **算法性能预测模型**
   - 图表14：专门进行算法性能预测模型建立，包括线性/多项式回归、模型评估和外推预测。。

> 数据来源：第 4 节的统计结果。


In [7]:
# -*- coding: utf-8 -*-
# 第五节：数据分析与可视化
# 生成输出目录：visualization_results/*
# 依赖：第四节 outputs_branch_bound_metrics/*

import os
import pandas as pd
import numpy as np
import warnings

# 首先设置Matplotlib
import matplotlib
matplotlib.use('Agg')  
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from scipy.optimize import curve_fit

warnings.filterwarnings('ignore')

# 重置matplotlib设置
plt.rcParams.update(plt.rcParamsDefault)

# 设置基本样式
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3
plt.rcParams['font.size'] = 10

# 定义数据目录
data_dir = "outputs_branch_bound_metrics/"

# 1. 加载所有CSV文件
print("正在加载数据文件...")
file_names = [
    "runtime_statistics.csv",
    "node_efficiency_statistics.csv",
    "solution_quality_statistics.csv",
    "convergence_speed_statistics.csv",
    "memory_consumption_statistics.csv",
    "statistical_significance_tests.csv",
    "comprehensive_summary.csv",
    "instance_type_performance_summary.csv",
    "performance_improvement_heatmap_data.csv"
]

# 创建数据字典存储所有数据
data = {}
for file_name in file_names:
    file_path = os.path.join(data_dir, file_name)
    try:
        df = pd.read_csv(file_path)
        data[file_name.replace('.csv', '')] = df
        print(f"✓ {file_name}: {df.shape} 形状")
    except Exception as e:
        print(f"✗ {file_name}: 加载失败 - {e}")

print(f"\n成功加载 {len(data)} 个数据文件")

# 创建输出目录用于保存图表
output_dir = "visualization_results"
os.makedirs(output_dir, exist_ok=True)

# 图表1: 运行时间分析
print("\n1. 生成运行时间分析图表...")
if 'runtime_statistics' in data:
    runtime_df = data['runtime_statistics']
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    algorithms = runtime_df['algo'].unique()
    colors = {'bb_basic': '#1f77b4', 'bb_enhanced': '#ff7f0e'}
    
    for algo in algorithms:
        algo_data = runtime_df[runtime_df['algo'] == algo].sort_values('n')
        ax1.plot(algo_data['n'], algo_data['mean_runtime_s'], 
                marker='o', linewidth=2, label=algo, color=colors[algo])
    
    ax1.set_xlabel('Problem Size (n)', fontsize=12)
    ax1.set_ylabel('Average Runtime (s)', fontsize=12)
    ax1.set_title('Average Runtime vs Problem Size', fontsize=14, fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_yscale('log')
    
    for algo in algorithms:
        algo_data = runtime_df[runtime_df['algo'] == algo].sort_values('n')
        ax2.plot(algo_data['n'], algo_data['mean_time_per_node'], 
                marker='s', linewidth=2, label=algo, color=colors[algo])
    
    ax2.set_xlabel('Problem Size (n)', fontsize=12)
    ax2.set_ylabel('Time per Node (s)', fontsize=12)
    ax2.set_title('Node Processing Efficiency', fontsize=14, fontweight='bold')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(f'{output_dir}/1_runtime_analysis.png', dpi=300, bbox_inches='tight')
    print("运行时间分析图表已保存")

# 图表2: 节点效率分析
print("\n2. 生成节点效率分析图表...")
if 'node_efficiency_statistics' in data:
    node_df = data['node_efficiency_statistics']
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    for algo in node_df['algo'].unique():
        algo_data = node_df[node_df['algo'] == algo].sort_values('n')
        ax1.plot(algo_data['n'], algo_data['mean_nodes_explored'], 
                marker='o', linewidth=2, label=algo)
    
    ax1.set_xlabel('Problem Size (n)', fontsize=12)
    ax1.set_ylabel('Average Nodes Explored', fontsize=12)
    ax1.set_title('Nodes Explored Comparison', fontsize=14, fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_yscale('log')
    
    for algo in node_df['algo'].unique():
        algo_data = node_df[node_df['algo'] == algo].sort_values('n')
        ax2.plot(algo_data['n'], algo_data['mean_exploration_ratio'], 
                marker='s', linewidth=2, label=algo)
    
    ax2.set_xlabel('Problem Size (n)', fontsize=12)
    ax2.set_ylabel('Exploration Ratio (%)', fontsize=12)
    ax2.set_title('Node Exploration Efficiency', fontsize=14, fontweight='bold')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(f'{output_dir}/2_node_efficiency.png', dpi=300, bbox_inches='tight')
    print("节点效率分析图表已保存")

# 图表3: 解质量分析
print("\n3. 生成解质量分析图表...")
if 'solution_quality_statistics' in data:
    quality_df = data['solution_quality_statistics']
    
    fig, ax = plt.subplots(figsize=(12, 8))
    
    basic_data = quality_df[quality_df['algo'] == 'bb_basic']
    enhanced_data = quality_df[quality_df['algo'] == 'bb_enhanced']
    
    instance_types = sorted(basic_data['class'].unique())
    basic_accuracy = []
    enhanced_accuracy = []
    
    for inst in instance_types:
        basic_val = basic_data[basic_data['class'] == inst]['accuracy_rate'].values[0]
        enhanced_val = enhanced_data[enhanced_data['class'] == inst]['accuracy_rate'].values[0]
        basic_accuracy.append(basic_val * 100)
        enhanced_accuracy.append(enhanced_val * 100)
    
    x = np.arange(len(instance_types))
    width = 0.35
    
    rects1 = ax.bar(x - width/2, basic_accuracy, width, 
                   label='bb_basic', alpha=0.8, color='#1f77b4')
    rects2 = ax.bar(x + width/2, enhanced_accuracy, width,
                   label='bb_enhanced', alpha=0.8, color='#ff7f0e')
    
    ax.set_xlabel('Instance Type', fontsize=12)
    ax.set_ylabel('Accuracy Rate (%)', fontsize=12)
    ax.set_title('Solution Quality Comparison', fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(instance_types, rotation=45, ha='right')
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')
    ax.set_ylim(99.5, 100.5)
    
    for rect in rects1:
        height = rect.get_height()
        ax.annotate(f'{height:.2f}%',
                   xy=(rect.get_x() + rect.get_width() / 2, height),
                   xytext=(0, 3), textcoords="offset points",
                   ha='center', va='bottom', fontsize=9)
    
    for rect in rects2:
        height = rect.get_height()
        ax.annotate(f'{height:.2f}%',
                   xy=(rect.get_x() + rect.get_width() / 2, height),
                   xytext=(0, 3), textcoords="offset points",
                   ha='center', va='bottom', fontsize=9)
    
    plt.tight_layout()
    plt.savefig(f'{output_dir}/3_solution_quality.png', dpi=300, bbox_inches='tight')
    print("解质量分析图表已保存")

# 图表4: 收敛速度分析
print("\n4. 生成收敛速度分析图表...")
if 'convergence_speed_statistics' in data:
    conv_df = data['convergence_speed_statistics']
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    for algo in conv_df['algo'].unique():
        algo_data = conv_df[conv_df['algo'] == algo].sort_values('n')
        ax1.plot(algo_data['n'], algo_data['mean_first_opt_node'], 
                marker='o', linewidth=2, label=algo)
    
    ax1.set_xlabel('Problem Size (n)', fontsize=12)
    ax1.set_ylabel('Average First Optimal Node', fontsize=12)
    ax1.set_title('First Optimal Node Comparison', fontsize=14, fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_yscale('log')
    
    for algo in conv_df['algo'].unique():
        algo_data = conv_df[conv_df['algo'] == algo].sort_values('n')
        ax2.plot(algo_data['n'], algo_data['mean_convergence_speed'], 
                marker='s', linewidth=2, label=algo)
    
    ax2.set_xlabel('Problem Size (n)', fontsize=12)
    ax2.set_ylabel('Average Convergence Speed Index', fontsize=12)
    ax2.set_title('Convergence Speed Index Comparison', fontsize=14, fontweight='bold')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(f'{output_dir}/4_convergence_speed.png', dpi=300, bbox_inches='tight')
    print("收敛速度分析图表已保存")

# 图表5: 内存消耗分析
print("\n5. 生成内存消耗分析图表...")
if 'memory_consumption_statistics' in data:
    mem_df = data['memory_consumption_statistics']
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    for algo in mem_df['algo'].unique():
        algo_data = mem_df[mem_df['algo'] == algo].sort_values('n')
        ax1.plot(algo_data['n'], algo_data['mean_memory_mb'], 
                marker='o', linewidth=2, label=algo)
    
    ax1.set_xlabel('Problem Size (n)', fontsize=12)
    ax1.set_ylabel('Average Memory Usage (MB)', fontsize=12)
    ax1.set_title('Average Memory Consumption', fontsize=14, fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    for algo in mem_df['algo'].unique():
        algo_data = mem_df[mem_df['algo'] == algo].sort_values('n')
        ax2.plot(algo_data['n'], algo_data['max_memory_mb'], 
                marker='s', linewidth=2, label=algo)
    
    ax2.set_xlabel('Problem Size (n)', fontsize=12)
    ax2.set_ylabel('Peak Memory Usage (MB)', fontsize=12)
    ax2.set_title('Peak Memory Consumption', fontsize=14, fontweight='bold')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(f'{output_dir}/5_memory_consumption.png', dpi=300, bbox_inches='tight')
    print("内存消耗分析图表已保存")

# 图表6: 统计显著性分析
print("\n6. 生成统计显著性分析图表...")
if 'statistical_significance_tests' in data:
    sig_df = data['statistical_significance_tests']
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    metrics = sig_df['metric'].tolist()
    # 将中文指标名转换为英文
    metric_dict = {'运行时间': 'Runtime', '探索节点数': 'Nodes Explored', 
                  '内存消耗': 'Memory Usage', '收敛速度': 'Convergence Speed'}
    metric_labels = [metric_dict.get(m, m) for m in metrics]
    
    improvements = sig_df['improvement_pct'].values
    p_values = sig_df['p_value'].values
    
    x = np.arange(len(metrics))
    width = 0.35
    
    # 使用不同颜色表示改进/退步
    bar_colors = ['#4CAF50' if imp > 0 else '#F44336' for imp in improvements]
    
    bars = ax.bar(x, improvements, width, color=bar_colors, alpha=0.8)
    
    ax.set_xlabel('Performance Metric', fontsize=12)
    ax.set_ylabel('Improvement (%)', fontsize=12)
    ax.set_title('Statistical Significance Tests\n(Positive = Enhancement is better)', 
                fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(metric_labels)
    ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    ax.grid(True, alpha=0.3, axis='y')
    
    # 添加p-value信息
    for i, (bar, p_val) in enumerate(zip(bars, p_values)):
        height = bar.get_height()
        sign = '***' if p_val < 0.001 else '**' if p_val < 0.01 else '*' if p_val < 0.05 else ''
        label = f'{height:.1f}%{sign}'
        va = 'bottom' if height >= 0 else 'top'
        y_offset = 0.5 if height >= 0 else -0.5
        ax.text(bar.get_x() + bar.get_width()/2, height + y_offset,
               label, ha='center', va=va, fontsize=10, fontweight='bold')
    
    plt.tight_layout()
    plt.savefig(f'{output_dir}/6_statistical_significance.png', dpi=300, bbox_inches='tight')
    print("版统计显著性分析图表已保存")

# 图表7: 综合性能对比雷达图
print("\n7. 生成综合性能对比雷达图...")
if 'comprehensive_summary' in data:
    summary_df = data['comprehensive_summary']
    
    metrics = ['平均运行时间(s)', '平均探索节点数', '精确率(%)', '平均收敛速度指数', '平均内存消耗(MB)']
    metric_names = ['Runtime (lower)', 'Nodes Explored (lower)', 'Accuracy (higher)', 
                   'Convergence Speed (higher)', 'Memory Usage (lower)']
    
    basic_row = summary_df[summary_df['算法'] == 'bb_basic'].iloc[0]
    enhanced_row = summary_df[summary_df['算法'] == 'bb_enhanced'].iloc[0]
    
    basic_values = [basic_row[m] for m in metrics]
    enhanced_values = [enhanced_row[m] for m in metrics]
    
    normalized_basic = []
    normalized_enhanced = []
    
    for i, (name, basic_val, enhanced_val) in enumerate(zip(metric_names, basic_values, enhanced_values)):
        if 'lower' in name:
            norm_basic = 1 / basic_val if basic_val > 0 else 0
            norm_enhanced = 1 / enhanced_val if enhanced_val > 0 else 0
        else:
            norm_basic = basic_val
            norm_enhanced = enhanced_val
        
        normalized_basic.append(norm_basic)
        normalized_enhanced.append(norm_enhanced)
    
    max_val = max(max(normalized_basic), max(normalized_enhanced))
    if max_val > 0:
        normalized_basic = [val/max_val for val in normalized_basic]
        normalized_enhanced = [val/max_val for val in normalized_enhanced]
    
    angles = np.linspace(0, 2*np.pi, len(metric_names), endpoint=False).tolist()
    angles += angles[:1]
    
    normalized_basic += normalized_basic[:1]
    normalized_enhanced += normalized_enhanced[:1]
    
    fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='polar'))
    
    ax.plot(angles, normalized_basic, 'o-', linewidth=2, label='bb_basic', color='#1f77b4')
    ax.fill(angles, normalized_basic, alpha=0.25, color='#1f77b4')
    
    ax.plot(angles, normalized_enhanced, 's-', linewidth=2, label='bb_enhanced', color='#ff7f0e')
    ax.fill(angles, normalized_enhanced, alpha=0.25, color='#ff7f0e')
    
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(metric_names, fontsize=10)
    ax.set_ylim(0, 1.1)
    ax.set_title('Comprehensive Performance Comparison\n(Normalized values, higher is better)', 
                fontsize=14, fontweight='bold', pad=20)
    ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))
    ax.grid(True)
    
    plt.tight_layout()
    plt.savefig(f'{output_dir}/7_radar_chart.png', dpi=300, bbox_inches='tight')
    print("综合性能对比雷达图已保存")

# 图表8: 性能改进热力图
print("\n8. 生成性能改进热力图...")
if 'performance_improvement_heatmap_data' in data:
    heatmap_df = data['performance_improvement_heatmap_data']
    
    fig, ax = plt.subplots(figsize=(12, 8))
    
    instance_types = heatmap_df['实例类型']
    if '时间改进(%)' in heatmap_df.columns:
        improvements = heatmap_df['时间改进(%)']
    else:
        improvements = np.zeros(len(instance_types))
    
    colors = ['#4CAF50' if x > 0 else '#F44336' for x in improvements]
    
    bars = ax.barh(instance_types, improvements, color=colors, alpha=0.8)
    
    ax.set_xlabel('Improvement (%)', fontsize=12)
    ax.set_ylabel('Instance Type', fontsize=12)
    ax.set_title('Time Improvement by Instance Type\n(Positive = Enhancement is better)', 
                fontsize=14, fontweight='bold')
    ax.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
    ax.grid(True, alpha=0.3, axis='x')
    
    for bar, improvement in zip(bars, improvements):
        width = bar.get_width()
        label = f'+{improvement:.1f}%' if improvement > 0 else f'{improvement:.1f}%'
        ha = 'left' if width >= 0 else 'right'
        x_pos = width + (1 if width >= 0 else -1)
        ax.text(x_pos, bar.get_y() + bar.get_height()/2,
               label, ha=ha, va='center', fontweight='bold', fontsize=10)
    
    plt.tight_layout()
    plt.savefig(f'{output_dir}/8_improvement_heatmap.png', dpi=300, bbox_inches='tight')
    print("性能改进热力图已保存")

# 图表9: 算法对比汇总
print("\n9. 生成算法对比汇总图表...")
if 'comprehensive_summary' in data:
    summary_df = data['comprehensive_summary']
    
    fig, ax = plt.subplots(figsize=(12, 8))
    
    key_metrics = ['平均运行时间(s)', '平均探索节点数', '精确率(%)', '平均内存消耗(MB)']
    metric_names = ['Runtime (s)', 'Nodes Explored', 'Accuracy (%)', 'Memory (MB)']
    
    basic_vals = []
    enhanced_vals = []
    
    for metric in key_metrics:
        basic_val = summary_df[summary_df['算法'] == 'bb_basic'][metric].values[0]
        enhanced_val = summary_df[summary_df['算法'] == 'bb_enhanced'][metric].values[0]
        basic_vals.append(basic_val)
        enhanced_vals.append(enhanced_val)
    
    x = np.arange(len(metric_names))
    width = 0.35
    
    rects1 = ax.bar(x - width/2, basic_vals, width, 
                    label='bb_basic', alpha=0.8, color='#1f77b4')
    rects2 = ax.bar(x + width/2, enhanced_vals, width,
                    label='bb_enhanced', alpha=0.8, color='#ff7f0e')
    
    ax.set_xlabel('Performance Metric', fontsize=12)
    ax.set_ylabel('Value', fontsize=12)
    ax.set_title('Key Performance Metrics Comparison\n(Note: Enhanced algorithm has higher overhead)', 
                fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(metric_names)
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')
    
    for i, (rect, val) in enumerate(zip(rects1, basic_vals)):
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2, height * 1.02,
               f'{val:.4f}' if val < 0.01 else f'{val:.2f}', 
               ha='center', va='bottom', fontsize=9)
    
    for i, (rect, val) in enumerate(zip(rects2, enhanced_vals)):
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2, height * 1.02,
               f'{val:.4f}' if val < 0.01 else f'{val:.2f}', 
               ha='center', va='bottom', fontsize=9)
    
    plt.tight_layout()
    plt.savefig(f'{output_dir}/9_algorithm_comparison_fixed.png', dpi=300, bbox_inches='tight')
    print("算法对比汇总图表已保存")

# 图表10: 增强算法在特定实例中的节点探索优势
print("\n10. 生成增强算法在特定实例中的节点探索优势图表...")

advantage_instances = ['ProbingBenefit', 'Uncorrelated', 'WeaklyCorr']
node_reduction = [13.9, 11.5, 30.5]  # 节点减少百分比
time_increase = [3.4, 3.0, 2.5]  # 时间增加倍数

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# 左侧：节点探索减少
bars1 = ax1.bar(advantage_instances, node_reduction, 
                color=['#4CAF50', '#4CAF50', '#4CAF50'], alpha=0.8)
ax1.set_xlabel('Instance Type', fontsize=12)
ax1.set_ylabel('Node Exploration Reduction (%)', fontsize=12)
ax1.set_title('Enhanced Algorithm: Node Exploration Advantage\n(Positive = Fewer nodes explored)', 
             fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3, axis='y')

for bar, reduction in zip(bars1, node_reduction):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2, height + 0.5,
            f'{reduction:.1f}%', ha='center', va='bottom', 
            fontweight='bold', fontsize=11)

# 右侧：时间增加倍数
bars2 = ax2.bar(advantage_instances, time_increase, 
                color=['#F44336', '#F44336', '#F44336'], alpha=0.8)
ax2.set_xlabel('Instance Type', fontsize=12)
ax2.set_ylabel('Time Increase (Multiple)', fontsize=12)
ax2.set_title('Enhanced Algorithm: Computational Overhead\n(Trade-off: More time per node)', 
             fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='y')

for bar, increase in zip(bars2, time_increase):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2, height + 0.1,
            f'{increase:.1f}x', ha='center', va='bottom', 
            fontweight='bold', fontsize=11)

plt.suptitle('Enhanced Algorithm: Trade-off Analysis\nNode Exploration Reduction vs Computational Overhead', 
            fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig(f'{output_dir}/10_enhanced_advantage_nodes.png', dpi=300, bbox_inches='tight')
print("增强算法节点探索优势图表已保存")

# 图表11: 增强算法的探测技术效果分析
print("\n11. 生成增强算法的探测技术效果分析图表...")

fig, ax = plt.subplots(figsize=(12, 8))

# 创建更详细的性能对比
instances = advantage_instances
x = np.arange(len(instances))
width = 0.2

basic_nodes = [634.5, 66.4, 225.2]  # 基础算法探索节点数
enhanced_nodes = [546.5, 58.8, 156.6]  # 增强算法探索节点数
basic_time = [0.0040, 0.00043, 0.00125]  # 基础算法运行时间(s)
enhanced_time = [0.0135, 0.00132, 0.00313]  # 增强算法运行时间(s)

# 创建双轴图表
ax1 = ax
bars1 = ax1.bar(x - width, basic_nodes, width, label='bb_basic (Nodes)', 
                alpha=0.8, color='#1f77b4')
bars2 = ax1.bar(x, enhanced_nodes, width, label='bb_enhanced (Nodes)', 
                alpha=0.8, color='#ff7f0e')
ax1.set_xlabel('Instance Type', fontsize=12)
ax1.set_ylabel('Nodes Explored', fontsize=12, color='black')
ax1.tick_params(axis='y', labelcolor='black')
ax1.set_xticks(x)
ax1.set_xticklabels(instances)

# 添加运行时间的第二个y轴
ax2 = ax1.twinx()
ax2.plot(x, basic_time, 'o-', label='bb_basic (Time)', 
         color='#1f77b4', linewidth=2, markersize=8)
ax2.plot(x, enhanced_time, 's-', label='bb_enhanced (Time)', 
         color='#ff7f0e', linewidth=2, markersize=8)
ax2.set_ylabel('Runtime (seconds)', fontsize=12, color='black')
ax2.tick_params(axis='y', labelcolor='black')

# 合并图例
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

ax1.set_title('Probing Technique Effectiveness Analysis\n(Enhanced algorithm explores fewer nodes but requires more time per node)', 
             fontsize=14, fontweight='bold', pad=20)
ax1.grid(True, alpha=0.3, axis='y')

# 添加改进百分比标注
for i, (basic_n, enhanced_n) in enumerate(zip(basic_nodes, enhanced_nodes)):
    reduction = (basic_n - enhanced_n) / basic_n * 100
    ax1.text(x[i], max(basic_n, enhanced_n) * 1.05, 
            f'↓{reduction:.1f}%', ha='center', va='bottom', 
            fontweight='bold', fontsize=10, color='green')

plt.tight_layout()
plt.savefig(f'{output_dir}/11_probing_effectiveness.png', dpi=300, bbox_inches='tight')
print("探测技术效果分析图表已保存")

# 新图表12: 增强算法的综合评估
print("\n12. 生成增强算法的综合评估图表...")

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 子图1: 节点探索效率改进
ax1 = axes[0, 0]
ax1.barh(advantage_instances, node_reduction, 
         color=['#4CAF50', '#4CAF50', '#4CAF50'], alpha=0.8)
ax1.set_xlabel('Node Reduction (%)', fontsize=11)
ax1.set_title('Node Exploration Efficiency Improvement', fontsize=12, fontweight='bold')
ax1.grid(True, alpha=0.3, axis='x')

for i, (inst, reduction) in enumerate(zip(advantage_instances, node_reduction)):
    ax1.text(reduction + 0.5, i, f'{reduction:.1f}%', 
            va='center', fontweight='bold', fontsize=10)

# 子图2: 每节点处理时间增加
ax2 = axes[0, 1]
time_per_node_increase = [2.5, 2.8, 2.2]  # 估计的每节点时间增加倍数
ax2.barh(advantage_instances, time_per_node_increase, 
         color=['#F44336', '#F44336', '#F44336'], alpha=0.8)
ax2.set_xlabel('Time per Node Increase (Multiple)', fontsize=11)
ax2.set_title('Computational Overhead per Node', fontsize=12, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='x')

for i, (inst, increase) in enumerate(zip(advantage_instances, time_per_node_increase)):
    ax2.text(increase + 0.1, i, f'{increase:.1f}x', 
            va='center', fontweight='bold', fontsize=10)

# 子图3: 改进效果与实例类型关系
ax3 = axes[1, 0]
instance_categories = ['ProbingBenefit', 'Uncorrelated', 'WeaklyCorr', 'Other']
improvement_rates = [13.9, 11.5, 30.5, -5.0]  # 其他实例平均下降5%
colors = ['#4CAF50', '#4CAF50', '#4CAF50', '#F44336']
ax3.bar(instance_categories, improvement_rates, color=colors, alpha=0.8)
ax3.set_xlabel('Instance Category', fontsize=11)
ax3.set_ylabel('Node Improvement (%)', fontsize=11)
ax3.set_title('Improvement Pattern by Instance Category', fontsize=12, fontweight='bold')
ax3.set_xticklabels(instance_categories, rotation=45, ha='right')
ax3.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax3.grid(True, alpha=0.3, axis='y')

for i, (cat, rate) in enumerate(zip(instance_categories, improvement_rates)):
    ax3.text(i, rate + (1 if rate > 0 else -1), f'{rate:.1f}%', 
            ha='center', va='bottom' if rate > 0 else 'top', 
            fontweight='bold', fontsize=10)

# 子图4: 总结评估
ax4 = axes[1, 1]
assessment_criteria = ['Node Exploration', 'Solution Quality', 'Convergence', 'Memory Usage']
enhanced_scores = [7, 10, 6, 4]  # 1-10分评分
basic_scores = [6, 10, 8, 9]

x = np.arange(len(assessment_criteria))
width = 0.35
ax4.bar(x - width/2, basic_scores, width, label='bb_basic', alpha=0.8, color='#1f77b4')
ax4.bar(x + width/2, enhanced_scores, width, label='bb_enhanced', alpha=0.8, color='#ff7f0e')
ax4.set_xlabel('Assessment Criteria', fontsize=11)
ax4.set_ylabel('Score (1-10)', fontsize=11)
ax4.set_title('Overall Algorithm Assessment', fontsize=12, fontweight='bold')
ax4.set_xticks(x)
ax4.set_xticklabels(assessment_criteria)
ax4.set_ylim(0, 11)
ax4.legend()
ax4.grid(True, alpha=0.3, axis='y')

plt.suptitle('Enhanced Algorithm: Comprehensive Performance Evaluation', 
            fontsize=16, fontweight='bold', y=0.98)
plt.tight_layout()
plt.savefig(f'{output_dir}/12_comprehensive_evaluation.png', dpi=300, bbox_inches='tight')
print("增强算法综合评估图表已保存")

# 图表13: 算法复杂度增长趋势分析
print("\n13. 生成算法复杂度增长趋势分析图表...")

# 辅助函数：格式化多项式方程为字符串
def format_polynomial_equation(coeffs):
    """
    格式化多项式方程为易读的字符串
    输入: 多项式系数数组 [a_n, a_{n-1}, ..., a_1, a_0]
    输出: 格式化的方程字符串
    """
    terms = []
    degree = len(coeffs) - 1
    
    for i, coeff in enumerate(coeffs):
        power = degree - i
        
        if abs(coeff) < 1e-10:  # 忽略接近零的系数
            continue
            
        # 格式化系数
        if power == 0:
            term = f"{coeff:.2e}"
        elif power == 1:
            term = f"{coeff:.2e}·n"
        else:
            term = f"{coeff:.2e}·n^{power}"
        
        terms.append(term)
    
    if not terms:
        return "0"
    
    return " + ".join(terms).replace("+ -", "- ")

if 'runtime_statistics' in data:
    runtime_df = data['runtime_statistics']
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 提取两种算法的数据
    basic_times = runtime_df[runtime_df['algo'] == 'bb_basic'].sort_values('n')
    enhanced_times = runtime_df[runtime_df['algo'] == 'bb_enhanced'].sort_values('n')
    
    # 对同一问题规模下的多个实例取平均值
    basic_times_grouped = basic_times.groupby('n')['mean_runtime_s'].mean().reset_index()
    enhanced_times_grouped = enhanced_times.groupby('n')['mean_runtime_s'].mean().reset_index()
    
    # 子图1: 实际运行时间增长趋势
    ax1 = axes[0, 0]
    
    # 绘制实际运行时间（使用平均值）
    ax1.plot(basic_times_grouped['n'], basic_times_grouped['mean_runtime_s'], 
            'o-', label='bb_basic (Average)', color='#1f77b4', linewidth=2, markersize=8)
    ax1.plot(enhanced_times_grouped['n'], enhanced_times_grouped['mean_runtime_s'], 
            's-', label='bb_enhanced (Average)', color='#ff7f0e', linewidth=2, markersize=8)
    
    #基于实际数据的曲线拟合
    from scipy.optimize import curve_fit
    
    # 定义拟合函数
    def exp_func(x, a, b):
        return a * np.exp(b * x)
    
    # 使用基础算法数据进行拟合
    n_vals = basic_times_grouped['n'].values
    time_vals = basic_times_grouped['mean_runtime_s'].values
    
    # 指数拟合
    try:
        # 排除零值和异常值
        valid_mask = (time_vals > 0) & (time_vals < np.percentile(time_vals, 95))
        if np.sum(valid_mask) >= 3:
            popt_exp, _ = curve_fit(exp_func, n_vals[valid_mask], 
                                  time_vals[valid_mask], 
                                  p0=[1e-6, 0.3], maxfev=5000)
            
            # 生成拟合曲线
            n_smooth = np.linspace(n_vals.min(), n_vals.max(), 100)
            exp_fit = exp_func(n_smooth, *popt_exp)
            
            # 计算R²值
            residuals = time_vals - exp_func(n_vals, *popt_exp)
            ss_res = np.sum(residuals**2)
            ss_tot = np.sum((time_vals - np.mean(time_vals))**2)
            r_squared = 1 - (ss_res / ss_tot)
            
            # 更新灰色参考线
            ax1.plot(n_smooth, exp_fit, '--', 
                    label=f'Exponential trend (R²={r_squared:.3f})', 
                    color='gray', linewidth=2, alpha=0.7)
    except Exception as e:
        print(f"指数拟合失败: {e}")
        # 使用原始估计作为备选
        n_values = basic_times_grouped['n'].values
        exp_fit = 0.00001 * np.exp(0.15 * n_values)
        ax1.plot(n_values, exp_fit, '--', label='Estimated exponential trend', 
                color='gray', alpha=0.7)
    
    ax1.set_xlabel('Problem Size (n)', fontsize=12)
    ax1.set_ylabel('Average Runtime (seconds)', fontsize=12)
    ax1.set_title('Algorithm Runtime Growth\n(Average over 6 instances per n)', fontsize=14, fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_yscale('log')
    
    # 子图2: 运行时间与问题规模的回归分析（两种算法都分析）
    ax2 = axes[0, 1]
    
    # 对基础算法进行多项式回归
    X_basic = basic_times_grouped['n'].values.reshape(-1, 1)
    y_basic = basic_times_grouped['mean_runtime_s'].values
    
    # 对增强算法进行多项式回归
    X_enhanced = enhanced_times_grouped['n'].values.reshape(-1, 1)
    y_enhanced = enhanced_times_grouped['mean_runtime_s'].values
    
    # 尝试多种回归模型（1-3次多项式）
    degrees = [1, 2, 3]
    basic_colors = ['#1f77b4', '#2ca02c', '#d62728']
    enhanced_colors = ['#ff7f0e', '#9467bd', '#8c564b']
    basic_linestyles = ['--', '--', '--']
    enhanced_linestyles = [':', ':', ':']
    
    # 存储回归结果，用于打印
    regression_results = {'bb_basic': {}, 'bb_enhanced': {}}
    
    for i, degree in enumerate(degrees):
        # 基础算法回归
        coeffs_basic = np.polyfit(X_basic.flatten(), y_basic, degree)
        poly_func_basic = np.poly1d(coeffs_basic)
        X_smooth = np.linspace(min(X_basic.min(), X_enhanced.min()), 
                              max(X_basic.max(), X_enhanced.max()), 100)
        y_pred_basic = poly_func_basic(X_smooth)
        
        # 计算R²
        y_pred_actual = poly_func_basic(X_basic.flatten())
        ss_res = np.sum((y_basic - y_pred_actual) ** 2)
        ss_tot = np.sum((y_basic - np.mean(y_basic)) ** 2)
        r_squared = 1 - (ss_res / ss_tot) if ss_tot != 0 else 0
        
        # 存储结果
        regression_results['bb_basic'][degree] = {
            'coefficients': coeffs_basic,
            'r_squared': r_squared,
            'equation': format_polynomial_equation(coeffs_basic)
        }
        
        ax2.plot(X_smooth, y_pred_basic, basic_linestyles[i], 
                label=f'bb_basic (deg={degree}, R²={r_squared:.3f})', 
                color=basic_colors[i], alpha=0.7, linewidth=2)
        
        # 增强算法回归（如果数据点足够）
        if len(X_enhanced) >= degree + 1:
            coeffs_enhanced = np.polyfit(X_enhanced.flatten(), y_enhanced, degree)
            poly_func_enhanced = np.poly1d(coeffs_enhanced)
            y_pred_enhanced = poly_func_enhanced(X_smooth)
            
            # 计算R²
            y_pred_actual_enh = poly_func_enhanced(X_enhanced.flatten())
            ss_res_enh = np.sum((y_enhanced - y_pred_actual_enh) ** 2)
            ss_tot_enh = np.sum((y_enhanced - np.mean(y_enhanced)) ** 2)
            r_squared_enh = 1 - (ss_res_enh / ss_tot_enh) if ss_tot_enh != 0 else 0
            
            # 存储结果
            regression_results['bb_enhanced'][degree] = {
                'coefficients': coeffs_enhanced,
                'r_squared': r_squared_enh,
                'equation': format_polynomial_equation(coeffs_enhanced)
            }
            
            ax2.plot(X_smooth, y_pred_enhanced, enhanced_linestyles[i], 
                    label=f'bb_enhanced (deg={degree}, R²={r_squared_enh:.3f})', 
                    color=enhanced_colors[i], alpha=0.7, linewidth=2)
    
    # 绘制实际数据点
    ax2.scatter(X_basic, y_basic, label='bb_basic (Data)', 
               color='#1f77b4', s=60, alpha=0.7, edgecolor='black')
    ax2.scatter(X_enhanced, y_enhanced, label='bb_enhanced (Data)', 
               color='#ff7f0e', s=60, alpha=0.7, edgecolor='black')
    
    ax2.set_xlabel('Problem Size (n)', fontsize=12)
    ax2.set_ylabel('Runtime (seconds)', fontsize=12)
    ax2.set_title('Polynomial Regression Models for Both Algorithms', fontsize=14, fontweight='bold')
    ax2.legend(loc='upper left', fontsize=9)
    ax2.grid(True, alpha=0.3)
    ax2.set_yscale('log')
    
    # 在图上添加回归方程文本（简化版，只显示二次多项式）
    if 2 in regression_results['bb_basic']:
        eq_basic = regression_results['bb_basic'][2]['equation']
        r2_basic = regression_results['bb_basic'][2]['r_squared']
        eq_enhanced = regression_results['bb_enhanced'][2]['equation']
        r2_enhanced = regression_results['bb_enhanced'][2]['r_squared']
        
        textstr = '\n'.join((
            f'bb_basic (quadratic):',
            f'  time(n) = {eq_basic}',
            f'  R² = {r2_basic:.3f}',
            f'bb_enhanced (quadratic):',
            f'  time(n) = {eq_enhanced}',
            f'  R² = {r2_enhanced:.3f}'
        ))
        
        # 放置文本框在图的右上方
        ax2.text(0.98, 0.02, textstr, transform=ax2.transAxes, fontsize=8,
                verticalalignment='bottom', horizontalalignment='right',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5),
                family='monospace')
    
    # 输出回归方程到控制台
    print("\n" + "="*80)
    print("回归模型参数详细分析:")
    print("="*80)
    for algo in ['bb_basic', 'bb_enhanced']:
        print(f"\n{algo}:")
        for degree in degrees:
            if degree in regression_results[algo]:
                coeffs = regression_results[algo][degree]['coefficients']
                eq = regression_results[algo][degree]['equation']
                r2 = regression_results[algo][degree]['r_squared']
                print(f"  deg={degree}: time(n) = {eq} (R²={r2:.6f})")
                print(f"        系数: {coeffs}")
    
    # 子图3: 偏差分析 - 每节点处理时间
    ax3 = axes[1, 0]
    
    # 计算每节点处理时间的变化
    if 'node_efficiency_statistics' in data:
        basic_nodes = data['node_efficiency_statistics']
        basic_nodes_data = basic_nodes[basic_nodes['algo'] == 'bb_basic'].sort_values('n')
        enhanced_nodes_data = basic_nodes[basic_nodes['algo'] == 'bb_enhanced'].sort_values('n')
        
        # 对节点数据也进行分组平均
        basic_nodes_grouped = basic_nodes_data.groupby('n')['mean_nodes_explored'].mean().reset_index()
        enhanced_nodes_grouped = enhanced_nodes_data.groupby('n')['mean_nodes_explored'].mean().reset_index()
        
        # 确保时间数据和节点数据的n值对齐
        basic_merged = pd.merge(basic_times_grouped, basic_nodes_grouped, on='n')
        enhanced_merged = pd.merge(enhanced_times_grouped, enhanced_nodes_grouped, on='n')
        
        # 计算每节点时间（使用平均值）
        time_per_node_basic = basic_merged['mean_runtime_s'] / basic_merged['mean_nodes_explored']
        time_per_node_enhanced = enhanced_merged['mean_runtime_s'] / enhanced_merged['mean_nodes_explored']
        
        ax3.plot(basic_merged['n'], time_per_node_basic, 'o-', 
                label='bb_basic (Time per node)', color='#1f77b4', linewidth=2)
        ax3.plot(enhanced_merged['n'], time_per_node_enhanced, 's-', 
                label='bb_enhanced (Time per node)', color='#ff7f0e', linewidth=2)
    
    ax3.set_xlabel('Problem Size (n)', fontsize=12)
    ax3.set_ylabel('Time per Node (seconds)', fontsize=12)
    ax3.set_title('Node Processing Efficiency\n(Average time per node)', fontsize=14, fontweight='bold')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # 子图4: 不同实例类型的复杂度特征
    ax4 = axes[1, 1]

    if 'instance_type_performance_summary' in data:
        instance_df = data['instance_type_performance_summary']

        # 分组计算平均值
        instance_groups = instance_df.groupby('实例类型')

        categories = []
        avg_times_basic = []
        avg_times_enhanced = []

        for inst, group in instance_groups:
            categories.append(inst)
            basic_time = group[group['算法'] == 'bb_basic']['平均运行时间(s)'].mean()
            enhanced_time = group[group['算法'] == 'bb_enhanced']['平均运行时间(s)'].mean()
            avg_times_basic.append(basic_time)
            avg_times_enhanced.append(enhanced_time)

        x = np.arange(len(categories))
        width = 0.35

        bars1 = ax4.bar(x - width / 2, avg_times_basic, width,
                        label='bb_basic', alpha=0.8, color='#1f77b4')
        bars2 = ax4.bar(x + width / 2, avg_times_enhanced, width,
                        label='bb_enhanced', alpha=0.8, color='#ff7f0e')

        ax4.set_xlabel('Instance Type (by complexity)', fontsize=12)
        ax4.set_ylabel('Average Runtime (seconds)', fontsize=12)
        ax4.set_title('Algorithm Performance by Instance Complexity', fontsize=14, fontweight='bold')
        ax4.set_xticks(x)
        ax4.set_xticklabels(categories, rotation=45, ha='right')
        ax4.legend()
        ax4.grid(True, alpha=0.3, axis='y')
        ax4.set_yscale('log')
    
    plt.suptitle('Algorithm Complexity Analysis: Theory vs Experiment', 
                fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.savefig(f'{output_dir}/13_complexity_analysis.png', dpi=300, bbox_inches='tight')
    print("complexity analysis chart saved")

#补充图表14: 算法性能预测模型
print("\n14. 生成算法性能预测模型图表...")

if 'runtime_statistics' in data:
    runtime_df = data['runtime_statistics']
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 子图1: 指数回归模型
    ax1 = axes[0, 0]
    
    basic_data = runtime_df[runtime_df['algo'] == 'bb_basic'].sort_values('n')
    X = basic_data['n'].values.reshape(-1, 1)
    y = basic_data['mean_runtime_s'].values
    
    # 对对数变换后的数据进行线性回归
    log_y = np.log(y + 1e-10)
    
    lr = LinearRegression()
    lr.fit(X, log_y)
    log_y_pred = lr.predict(X)
    y_pred = np.exp(log_y_pred)
    
    ax1.scatter(X, y, label='Actual data', color='#1f77b4', s=50, alpha=0.7)
    ax1.plot(X, y_pred, 'r-', label=f'Exponential model (R²={lr.score(X, log_y):.3f})', 
            linewidth=2, color='#d62728')
    
    ax1.set_xlabel('Problem Size (n)', fontsize=12)
    ax1.set_ylabel('Runtime (seconds)', fontsize=12)
    ax1.set_title('Exponential Regression Model (Log scale)', fontsize=14, fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_yscale('log')
    
    # 子图2: 多项式回归模型比较
    ax2 = axes[0, 1]
    
    poly_degrees = [1, 2, 3, 4]
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
    
    for i, degree in enumerate(poly_degrees):
        poly = PolynomialFeatures(degree=degree)
        X_poly = poly.fit_transform(X)
        model = LinearRegression()
        model.fit(X_poly, y)
        y_pred = model.predict(X_poly)
        
        # 生成平滑曲线
        X_dense = np.linspace(X.min(), X.max(), 100).reshape(-1, 1)
        X_poly_dense = poly.transform(X_dense)
        y_pred_dense = model.predict(X_poly_dense)
        
        ax2.plot(X_dense, y_pred_dense, '--', 
                label=f'Polynomial deg={degree} (R²={model.score(X_poly, y):.3f})',
                color=colors[i], alpha=0.8, linewidth=2)
    
    ax2.scatter(X, y, label='Actual data', color='black', s=50, alpha=0.7)
    ax2.set_xlabel('Problem Size (n)', fontsize=12)
    ax2.set_ylabel('Runtime (seconds)', fontsize=12)
    ax2.set_title('Polynomial Regression Models Comparison', fontsize=14, fontweight='bold')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    ax2.set_yscale('log')
    
    # 子图3: 外推预测
    ax3 = axes[1, 0]
    
    # 使用最佳模型进行预测
    best_degree = 3
    poly = PolynomialFeatures(degree=best_degree)
    X_poly = poly.fit_transform(X)
    model = LinearRegression()
    model.fit(X_poly, y)
    
    # 外推到更大的n值
    n_extended = np.linspace(X.min(), X.max() * 1.5, 50).reshape(-1, 1)
    X_poly_extended = poly.transform(n_extended)
    y_extended = model.predict(X_poly_extended)
    
    ax3.scatter(X, y, label='Experimental data', color='#1f77b4', s=50, alpha=0.7)
    ax3.plot(n_extended, y_extended, 'r--', 
            label=f'Extrapolation (deg={best_degree})', linewidth=2, color='#d62728')
    ax3.axvline(x=X.max(), color='gray', linestyle=':', alpha=0.7, 
               label='Data boundary')
    ax3.fill_between(n_extended.flatten(), 0, y_extended, 
                     where=(n_extended.flatten() > X.max()), 
                     alpha=0.2, color='red', label='Extrapolation region')
    
    ax3.set_xlabel('Problem Size (n)', fontsize=12)
    ax3.set_ylabel('Predicted Runtime (seconds)', fontsize=12)
    ax3.set_title('Performance Extrapolation Prediction\n(For untested problem sizes)', fontsize=14, fontweight='bold')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    ax3.set_yscale('log')
    
    # 子图4: 预测模型评估
    ax4 = axes[1, 1]
    
    # 计算不同模型的预测误差
    models = ['Linear', 'Poly deg=2', 'Poly deg=3', 'Poly deg=4']
    mse_values = []
    r2_values = []
    
    for degree in [1, 2, 3, 4]:
        poly = PolynomialFeatures(degree=degree)
        X_poly = poly.fit_transform(X)
        model = LinearRegression()
        model.fit(X_poly, y)
        y_pred = model.predict(X_poly)
        
        mse = np.mean((y - y_pred) ** 2)
        r2 = model.score(X_poly, y)
        
        mse_values.append(mse)
        r2_values.append(r2)
    
    x_model = np.arange(len(models))
    width = 0.35
    
    bars1 = ax4.bar(x_model - width/2, mse_values, width, 
                   label='MSE (Mean Squared Error)', alpha=0.8, color='#1f77b4')
    ax4_twin = ax4.twinx()
    bars2 = ax4_twin.bar(x_model + width/2, r2_values, width,
                        label='R² Score', alpha=0.8, color='#ff7f0e')
    
    ax4.set_xlabel('Prediction Model', fontsize=12)
    ax4.set_ylabel('MSE', fontsize=12, color='#1f77b4')
    ax4_twin.set_ylabel('R² Score', fontsize=12, color='#ff7f0e')
    ax4.set_xticks(x_model)
    ax4.set_xticklabels(models, rotation=0)
    ax4.set_title('Prediction Model Performance Evaluation', fontsize=14, fontweight='bold')
    
    # 合并图例
    lines1, labels1 = ax4.get_legend_handles_labels()
    lines2, labels2 = ax4_twin.get_legend_handles_labels()
    ax4.legend(lines1 + lines2, labels1 + labels2, loc='upper left')
    
    plt.suptitle('Algorithm Performance Prediction Models', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.savefig(f'{output_dir}/14_performance_prediction.png', dpi=300, bbox_inches='tight')
    print("performance prediction chart saved")

#补充图表15: 综合性能边界总结表
print("\n15. 生成性能边界总结表图表...")

fig, ax = plt.subplots(figsize=(14, 8))

# 创建综合性能边界表
if 'comprehensive_summary' in data and 'instance_type_performance_summary' in data:
    summary_df = data['comprehensive_summary']
    instance_df = data['instance_type_performance_summary']
    
    # 准备性能边界数据（全部使用英文）
    performance_categories = [
        'Runtime (seconds)', 
        'Nodes Explored', 
        'Time per Node (seconds)', 
        'Memory Usage (MB)', 
        'Accuracy (%)',
        'Convergence Speed Index'
    ]
    
    # 从数据中提取最佳、最差情况
    performance_data = []
    
    # 运行时间
    basic_time_min = instance_df[instance_df['算法'] == 'bb_basic']['平均运行时间(s)'].min()
    basic_time_max = instance_df[instance_df['算法'] == 'bb_basic']['平均运行时间(s)'].max()
    enhanced_time_min = instance_df[instance_df['算法'] == 'bb_enhanced']['平均运行时间(s)'].min()
    enhanced_time_max = instance_df[instance_df['算法'] == 'bb_enhanced']['平均运行时间(s)'].max()
    
    performance_data.append([
        f"{basic_time_min:.6f} - {basic_time_max:.6f}",
        f"{enhanced_time_min:.6f} - {enhanced_time_max:.6f}"
    ])
    
    # 探索节点数
    basic_nodes_min = instance_df[instance_df['算法'] == 'bb_basic']['平均探索节点数'].min()
    basic_nodes_max = instance_df[instance_df['算法'] == 'bb_basic']['平均探索节点数'].max()
    enhanced_nodes_min = instance_df[instance_df['算法'] == 'bb_enhanced']['平均探索节点数'].min()
    enhanced_nodes_max = instance_df[instance_df['算法'] == 'bb_enhanced']['平均探索节点数'].max()
    
    performance_data.append([
        f"{basic_nodes_min:.1f} - {basic_nodes_max:.1f}",
        f"{enhanced_nodes_min:.1f} - {enhanced_nodes_max:.1f}"
    ])
    
    # 每节点时间（计算）
    basic_time_per_node = summary_df[summary_df['算法'] == 'bb_basic']['平均节点处理时间(s)'].values[0]
    enhanced_time_per_node = summary_df[summary_df['算法'] == 'bb_enhanced']['平均节点处理时间(s)'].values[0]
    
    performance_data.append([
        f"{basic_time_per_node:.8f}",
        f"{enhanced_time_per_node:.8f}"
    ])
    
    # 内存消耗
    basic_memory = summary_df[summary_df['算法'] == 'bb_basic']['平均内存消耗(MB)'].values[0]
    enhanced_memory = summary_df[summary_df['算法'] == 'bb_enhanced']['平均内存消耗(MB)'].values[0]
    
    performance_data.append([
        f"{basic_memory:.6f}",
        f"{enhanced_memory:.6f}"
    ])
    
    # 精确率
    basic_accuracy = summary_df[summary_df['算法'] == 'bb_basic']['精确率(%)'].values[0]
    enhanced_accuracy = summary_df[summary_df['算法'] == 'bb_enhanced']['精确率(%)'].values[0]
    
    performance_data.append([
        f"{basic_accuracy:.2f}%",
        f"{enhanced_accuracy:.2f}%"
    ])
    
    # 收敛速度指数
    basic_conv = summary_df[summary_df['算法'] == 'bb_basic']['平均收敛速度指数'].values[0]
    enhanced_conv = summary_df[summary_df['算法'] == 'bb_enhanced']['平均收敛速度指数'].values[0]
    
    performance_data.append([
        f"{basic_conv:.6f}",
        f"{enhanced_conv:.6f}"
    ])
    
    # 创建表格
    table = ax.table(cellText=performance_data,
                     rowLabels=performance_categories,
                     colLabels=['bb_basic', 'bb_enhanced'],
                     cellLoc='center',
                     loc='center',
                     colWidths=[0.25, 0.25])
    
    # 美化表格
    table.auto_set_font_size(False)
    table.set_fontsize(11)
    table.scale(1, 2)
    
    # 添加颜色
    for i in range(len(performance_categories)):
        for j in range(2):
            cell = table[(i+1, j)]
            if i == 0:  # Runtime
                if j == 0:  # basic
                    cell.set_facecolor('#E8F4FD')  # Light blue
                else:  # enhanced
                    cell.set_facecolor('#FFF0E8')  # Light orange
            elif i == 1:  # Nodes
                if j == 0:  # basic
                    cell.set_facecolor('#E8F4FD')
                else:  # enhanced
                    cell.set_facecolor('#FFF0E8')
            elif i == 2:  # Time per node
                if j == 0:  # basic
                    cell.set_facecolor('#F0F8FF')  # Very light blue
                else:  # enhanced
                    cell.set_facecolor('#FFF5EE')  # Very light orange
            elif i == 4:  # Accuracy (good for both)
                cell.set_facecolor('#F0FFF0')  # Light green
            elif i == 5:  # Convergence speed
                if j == 0:  # basic
                    cell.set_facecolor('#F5F5F5')  # Light gray
                else:  # enhanced
                    cell.set_facecolor('#FFF5F5')  # Light pink
    
    ax.axis('off')
    ax.set_title('Algorithm Performance Boundary Summary\n(Best-Worst Performance Ranges)', fontsize=16, fontweight='bold', pad=20)
    
    # 添加图例说明（英文）
    legend_text = (
        "Performance Boundary Explanation:\n"
        "• Runtime: Lower is better\n"
        "• Nodes Explored: Lower is better\n"
        "• Time per Node: Lower is better\n"
        "• Memory Usage: Lower is better\n"
        "• Accuracy: Higher is better\n"
        "• Convergence Speed Index: Higher is better"
    )
    ax.text(0.02, 0.02, legend_text, transform=ax.transAxes, fontsize=10,
            verticalalignment='bottom', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    plt.tight_layout()
    plt.savefig(f'{output_dir}/15_performance_boundary.png', dpi=300, bbox_inches='tight')
    print("Fixed performance boundary summary chart saved")

正在加载数据文件...
✓ runtime_statistics.csv: (48, 9) 形状
✓ node_efficiency_statistics.csv: (48, 11) 形状
✓ solution_quality_statistics.csv: (12, 6) 形状
✓ convergence_speed_statistics.csv: (48, 10) 形状
✓ memory_consumption_statistics.csv: (48, 9) 形状
✓ statistical_significance_tests.csv: (4, 7) 形状
✓ comprehensive_summary.csv: (2, 11) 形状
✓ instance_type_performance_summary.csv: (12, 9) 形状
✓ performance_improvement_heatmap_data.csv: (6, 8) 形状

成功加载 9 个数据文件

1. 生成运行时间分析图表...
运行时间分析图表已保存

2. 生成节点效率分析图表...
节点效率分析图表已保存

3. 生成解质量分析图表...
解质量分析图表已保存

4. 生成收敛速度分析图表...
收敛速度分析图表已保存

5. 生成内存消耗分析图表...
内存消耗分析图表已保存

6. 生成统计显著性分析图表...
版统计显著性分析图表已保存

7. 生成综合性能对比雷达图...
综合性能对比雷达图已保存

8. 生成性能改进热力图...
性能改进热力图已保存

9. 生成算法对比汇总图表...
算法对比汇总图表已保存

10. 生成增强算法在特定实例中的节点探索优势图表...
增强算法节点探索优势图表已保存

11. 生成增强算法的探测技术效果分析图表...
探测技术效果分析图表已保存

12. 生成增强算法的综合评估图表...
增强算法综合评估图表已保存

13. 生成算法复杂度增长趋势分析图表...

回归模型参数详细分析:

bb_basic:
  deg=1: time(n) = 4.76e-04·n - 6.87e-03 (R²=0.949515)
        系数: [ 0.00047618 -0.00687487]
  deg=2: time(n) = 1.

## 6. 总结与展望

### 6.1 结论回顾与性能差异原因
- **两种算法与复杂度**
  - **bb_basic** 与 **bb_enhanced** 在最坏情况下的时间复杂度均为$O(2^n⋅n)$但实际性能受剪枝效果显著影响,空间复杂度为$O(n)$。
  - **bb_enhanced** 因维护优先队列和存储探测信息，内存开销通常更高。

- **运行时间/节点效率/内存/收敛速度四方面结论**
  1) 运行时间：增强算法在大多数实例类型上运行时间更长（平均增加4.3倍），主要原因是每节点处理时间显著增加（平均增加2.5倍）。但在ProbingBenefit、Uncorrelated、WeaklyCorr实例中，节点探索减少优势（分别减少13.9%、11.5%、30.5%）部分抵消了额外开销。
  2) 节点探索效率：增强算法在三种实例类型中成功减少了节点探索，验证了探测技术的有效性。但在GreedyTrap、StronglyCorr、SubsetSum实例中未能减少探索节点。
  3) 内存消耗：增强算法内存使用平均增加32倍，主要来自优先队列和探测数据的存储开销。
  4) 收敛速度：增强算法收敛速度指数平均降低57.3%，表明其需要更多节点才能找到最优解，这与最佳上界优先策略的预期效果不符。

- **为何会出现差异**
 - **探测技术的双重效应**：探测技术在某些实例中提前识别无效分支，减少节点探索；但在其他实例中，探测计算本身增加了每节点处理时间，且可能错过最优路径。
 - **实例结构敏感性**：增强算法在ProbingBenefit、Uncorrelated、WeaklyCorr实例中表现相对较好，因为这些实例的特征更符合探测技术的设计假设。
 - **实现开销与收益失衡**：探测技术的计算开销超过了其带来的节点减少收益，导致总运行时间增加。
 - **内存与计算权衡**：增强算法用更高的内存消耗换取更智能的节点选择，但当前实现中内存开销过大

### 6.2 理论与实验一致性验证
 - **复杂度趋势验证**：两种算法实际运行时间均呈指数增长趋势，与分支限界法的理论复杂度一致。

 - **偏差来源分析**：
  1) 实现差异：Python解释器开销放大了每节点处理时间的差异，增强了算法中额外的探测计算在解释型语言中代价较高。
  2) 缓存效应：基础算法更简单的数据结构可能获得更好的缓存局部性。
  3) 随机性：实例生成中的随机性导致某些实例对探测技术更敏感。

 - **统计显著性**：t检验确认增强算法在运行时间、内存消耗和收敛速度方面显著劣于基础算法（p<0.001），但在节点探索数方面的改善不显著（p>0.05）。

### 6.3 适用场景与实践建议
- **优先选择bb_enhanced的情形**
  - 当问题实例具有ProbingBenefit、Uncorrelated或WeaklyCorr特征时，可尝试使用增强算法。
  - 当内存资源充足且更关注节点探索效率而非总运行时间时。
- **bb_basic仍然更优的情形**
  - 大多数实际应用场景，特别是时间敏感的应用。
  - 内存资源受限的环境。
  - StronglyCorr、SubsetSum、GreedyTrap等实例类型。
- **工程与评估建议**
  - 自适应策略：基于实例特征自动选择算法变体，对探测友好的实例使用增强版本。
  - 优化实现：使用编译型语言重写探测逻辑，减少解释器开销。
  - 内存优化：实现更紧凑的数据结构存储探测信息。

### 6.4 局限与改进方向
- **实例覆盖有限**：仅测试了6种合成实例类型，缺乏真实世界数据的验证。
- **规模范围限制**：最大测试规模为500个物品，更大规模下的性能趋势需要验证。
- **参数调优不足**：探测技术的参数未进行系统调优。

### 6.5 改进方向与未来工作
- **算法改进**
  1) 开发自适应探测机制，根据搜索进度动态调整探测深度。
  2) 设计轻量级探测技术，减少每节点计算开销。
  3) 结合机器学习预测子问题潜力，替代启发式探测。
- **实验扩展**
  1) 在更大规模问题（1000+物品）上测试算法可扩展性。
  2) 增加更多实例类型，包括真实世界数据集。

- **理论研究**
  1) 建立探测技术的收益-开销分析模型。
  2) 将增强分支限界法应用于其他组合优化问题（如旅行商问题、调度问题）。

---

**总体结语**
本研究对基础分支限界法和增强分支限界法进行了系统性实证比较。虽然增强算法在节点探索效率方面显示出一定优势，但由于每节点处理时间的显著增加，整体运行时间反而劣于基础算法。这一发现揭示了算法优化中的一个重要问题：局部改进可能被额外开销抵消。对于实践者，建议在内存充足且实例特征适合的情况下尝试增强算法，但在大多数实际场景中，基础分支限界法仍是更可靠的选择。